In [ ]:
# ============================================================
# Cell 1  –  Leela‑scope Lightning Simulator  (tier 1 … 9)
# ============================================================
"""
Leela-Style Lightning Simulator (Tiered Realism v2)
==================================================

This cell synthesises multi-station lightning waveforms with progressive
realism, extending and refining the previous simulator.  It mirrors the
UK Met-Office “Leela” network plus optional European/Atlantic outliers,
and allows nine distinct difficulty tiers to stress-test detection and
denoising pipelines without downstream changes.

1. USER KNOBS
--------------
- SEED        : RNG seed for full reproducibility.
- duration_min: Storm duration in minutes (quiet prelude + active storm).
- scenario    : Radial distribution of flashes (‘near’=50 km, ‘medium’=200 km, ‘far’=600 km).
- DIFFICULTY  : 1 (toy baseline) … 9 (ultimate stress test).
- FS, BITS    : Sampling frequency & ADC bit depth—kept consistent with pipeline.

2. STATION GEOGRAPHY & TIERS
-----------------------------
- stations_real lists the precise lat/lon of each real “Leela” site + major
  European capitals (Madrid, Milan, Athens) + Atlantic/Arctic outposts (Azores,
  Reykjavik, Algiers).
- Tier ≤4 keeps only the core UK/IE/DE/NL sites (LER, DUB, LON, BER).
- Tier 5 adds Madrid, Amsterdam, Milan, Athens → larger network geometry.
- Tier ≥6 includes all listed stations, creating a continental/Atlantic array.

3. DIFFICULTY-DEPENDENT FLAGS
--------------------------------
A `flags` dict drives feature toggles by tier:
- **ic_cg_mix (≥2)**: Cloud-to-cloud vs cloud-to-ground ratio, stroke count & amplitude.
- **multipath (≥3)**: Secondary echoes (ground/ionosphere reflections).
- **coloured_noise (≥4)**: Per-station white noise & hum levels vary.
- **rfi_tones / sprite_ring / false_transient / clipping (≥5)**:
  narrowband RFI tones, late “sprite” echoes, random false transients,
  and occasional ADC saturation to mimic hardware clipping.
- **multi_cell (≥6)**: Multiple drifting storm cells via Poisson(2)+1.
- **skywave / bg_sferics (≥7)**: Long-range propagation with frequency-dependent
  attenuation (“Q-factor” filter) and very distant background sferics.
- **clock_skew / gain_drift / dropouts (≥8)**: Sample-rate jitter, slow sensor
  gain drift, occasional data dropouts.
- **low_snr / burst_diversity (≥9)**: Extra attenuation for faint strokes +
  occasional K-change or “V-shaped” waveform diversity.

4. TIMELINE & STORM CELLS
--------------------------
- Quiet prelude (`pre_sec`) drawn uniformly from 5–30 s.
- Active storm (`storm_sec`) = duration_min × 60 s.
- n_cells = 1 for tiers <6; else Poisson(2)+1 cluster centres.
- Each cluster drifts linearly at 5–15 m s⁻¹, creating spatially drifting
  flash origins and overlapping sferics.

5. FLASH & STROKE GENERATION
-----------------------------
- Inter-flash gaps: lognormal(μ=3,σ=1) seconds, denser if multi_cell.
- Flash geometry tied to current storm-cell centroid (plus random radial
  offset up to R0).
- Stroke count & amplitude scaling vary by IC/CG mix.
- True stroke times stored; earliest arrival per flash used for regression.

6. BURST SHAPE LIBRARY
-----------------------
- Base sferic: decaying sinusoid (f=3–9 kHz, τ=0.4–2 ms).
- Tier ≥9 adds occasional K-change shape (fast rise, exponential decay).
- Tier ≥7 applies distance-dependent “skywave” filtering in frequency
  domain: H(f)=exp(–α·dist·(f/8 kHz)²).

7. NOISE & DISTORTIONS
-----------------------
- Base noise: white Gaussian + 50 Hz hum.
- Coloured noise (≥4): each station draws its own levels for white & hum.
- RFI tones (≥5): maritime/aviation frequencies, random phase per chunk.
- False transients & clipping (≥5): mimic RF bursts & ADC overload.
- Clock skew (±15 ppm) and gain drift (<5%) slowly distort entire record (≥8).
- Dropouts (≥8): random 0.3 s zeroed chunks simulate data loss.
- Low-SNR strokes (≥9): extra 50% attenuation for the faintest events.

8. SYNTHESIS LOOP
------------------
- Chunk-based processing (20 s chunks) for memory efficiency.
- Noise → gain drift → bursts insertion → false transients → clipping
  → low-pass Butterworth filter → quantisation to int16.
- `quantized[nm]` holds final ADC output for each station.

9. GROUND TRUTH LABELS
-----------------------
- `station_truth[nm][window_idx] = True` if any burst falls in that window.
- `stroke_records`: one entry per (event, stroke, station) including:
  event_id, stroke index, flash_type, true lat/lon/time, sample & window idx, shape tag.
- `stroke_samples`: array of earliest arrival sample indices
  (from the first station in `station_order`) for regression/QC.

10. DATAFRAMES & CONVERTERS
----------------------------
- `df_wave`: columns `time_s` + one column per station, length N rows.
- `df_labels`: one row per burst_record, ready for pandas filtering/analysis.
- `df_to_quantized(df_wave)`: reconstructs raw numpy arrays for pipeline code.
- `df_labels_to_events(df_labels)`: regenerates `events` list for detectors.

11. SUMMARY OUTPUT
------------------
Prints tier, cell count, total flashes, strokes per station,
and shapes of the two DataFrames for immediate QA.

Weaknesses & Future Enhancements
---------------------------------
- Real ionospheric propagation & ground conductivity not modelled in detail.
- No VLF/VHF microstructure or polarity changes of real sferics.
- Does not simulate simultaneous overlapping storms with different
  lightning behaviors (e.g. mesoscale convective systems vs single cells).
- Noise remains additive; real-world noise can be non-stationary bursts
  and non-Gaussian.
- All code runs single-threaded for clarity; could be parallelised.

Use `DIFFICULTY` to stress-test detection, compression, ML and deep-learning
methods as noise environments and signal morphologies become progressively
challenging.  Downstream notebooks and scripts require no modifications
to incorporate these advanced realism tiers.
"""
# ------------------------------------------------------------
import math, random, numpy as np, pandas as pd
from scipy.signal import butter, filtfilt
# ------------------------------------------------------------
# 0)  USER KNOBS
# ------------------------------------------------------------
SEED          = 424242
duration_min  = 5                # storm time span
scenario      = 'medium'         # 'near' | 'medium' | 'far'
DIFFICULTY    = 9               # 1 … 9
FS            = 109_375          # Hz (keep for pipeline compatibility)
BITS, VREF    = 14, 1.0
random.seed(SEED); np.random.seed(SEED)
# ------------------------------------------------------------
# 1)  Station list  (Met‑Office “Leela” + Atlantic & N‑Africa extensions)
#     Coordinates are approximate centre‑points of station boroughs.
# ------------------------------------------------------------
stations_real = {
    # Core UK / NW‑Europe
    'LER': (60.15,  -1.13),   # Lerwick (UK)
    'INV': (57.48,  -4.23),   # Inverness
    'DUB': (53.35,  -6.26),   # Dublin (IE)
    'LON': (51.50,  -0.12),   # London (UK)
    'AMS': (52.37,   4.90),   # Amsterdam (NL)
    'BER': (52.52,  13.40),   # Berlin (DE)
    # Extended coverage
    'MAD': (40.42,  -3.70),   # Madrid (ES)
    'MIL': (45.47,   9.19),   # Milan (IT)
    'ATH': (37.98,  23.73),   # Athens (GR)
    'ALG': (36.75,   3.06),   # Algiers (DZ)
    'AZO': (38.73, -27.18),   # Azores (PT)
    'REY': (64.14, -21.94),   # Reykjavik (IS)
}
# Tier ≤4 → only first 4 stations, tier 5 → first 8, else all
if DIFFICULTY <= 4:   keep = ['LER','DUB','LON','BER']
elif DIFFICULTY <= 5: keep = ['LER','DUB','LON','BER','MAD','AMS','MIL','ATH']
else:                  keep = list(stations_real.keys())
stations = {k: dict(lat=stations_real[k][0], lon=stations_real[k][1])
            for k in keep}
station_order = list(stations.keys())
STN = station_order
# ------------------------------------------------------------
# 2)  Convenience functions
# ------------------------------------------------------------
def hav(lat1, lon1, lat2, lon2):
    R=6371.0
    φ1,φ2 = map(math.radians, (lat1,lat2))
    dφ=math.radians(lat2-lat1); dλ=math.radians(lon2-lon1)
    a=math.sin(dφ/2)**2+math.cos(φ1)*math.cos(φ2)*math.sin(dλ/2)**2
    return 2*R*math.asin(math.sqrt(a))

rng = np.random.RandomState(SEED)

# ------------------------------------------------------------
# 3)  Timeline & global arrays
# ------------------------------------------------------------
pre_sec   = rng.uniform(5, 30)
storm_sec = duration_min*60
total_sec = pre_sec + storm_sec
N         = int(total_sec*FS)

quantized      = {nm: np.zeros(N, np.int16) for nm in STN}
station_truth  = {nm: np.zeros(N//1024, bool) for nm in STN}
burst_book     = []
stroke_records = []
events         = []
stroke_samples = []         # earliest‑arrival sample index per stroke

# ------------------------------------------------------------
# 4)  Difficulty‑dependent helper flags
# ------------------------------------------------------------
flags = dict(
    ic_cg_mix           = DIFFICULTY >= 2,
    multipath           = DIFFICULTY >= 3,
    coloured_noise      = DIFFICULTY >= 4,
    rfi_tones           = DIFFICULTY >= 5,
    sprite_ring         = DIFFICULTY >= 5,
    false_transient     = DIFFICULTY >= 5,
    clipping            = DIFFICULTY >= 5,
    multi_cell          = DIFFICULTY >= 6,
    skywave             = DIFFICULTY >= 7,
    bg_sferics          = DIFFICULTY >= 7,
    clock_skew          = DIFFICULTY >= 8,
    gain_drift          = DIFFICULTY >= 8,
    dropouts            = DIFFICULTY >= 8,
    low_snr             = DIFFICULTY >= 9,
    burst_diversity     = DIFFICULTY >= 9,
)

# ------------------------------------------------------------
# 5)  Storm‑cell geometry
# ------------------------------------------------------------
lat0 = np.mean([v['lat'] for v in stations.values()])
lon0 = np.mean([v['lon'] for v in stations.values()])
R0   = {'near':50,'medium':200,'far':600}[scenario]

n_cells = 1 if not flags['multi_cell'] else rng.poisson(2)+1
cells=[]
for cid in range(n_cells):
    θ = rng.uniform(0,2*math.pi)
    rad = R0 * rng.uniform(0.3,1.0)
    lat_c = lat0 + (rad/111)*math.cos(θ)
    lon_c = lon0 + (rad/111)*math.sin(θ)/math.cos(math.radians(lat0))
    drift = rng.uniform(-0.15,0.15,2)    # deg per hour ≈ 5‑15 m s‑¹
    cells.append(dict(lat=lat_c, lon=lon_c, drift=drift))

# ------------------------------------------------------------
# 6)  Generate flashes & strokes
# ------------------------------------------------------------
wave_dur  = 0.04; wave_len = int(wave_dur*FS)
tv_burst  = np.arange(wave_len)/FS
rfi_freqs = [14400,15800,20100]
rfi_table = {f: np.sin(2*math.pi*f*tv_burst).astype(np.float32) for f in rfi_freqs}

eid=0; t=pre_sec
while True:
    # choose cell, update centroid by drift
    cell = cells[rng.randint(len(cells))]
    cell_age = t-pre_sec
    clat = cell['lat'] + cell['drift'][0]*cell_age/3600
    clon = cell['lon'] + cell['drift'][1]*cell_age/3600
    # flash schedule
    gap = rng.lognormal(3.0, 1.0)
    if flags['multi_cell']:
        gap *= rng.uniform(0.3,1.0)      # denser flashes
    t += gap
    if t >= total_sec: break
    eid+=1
    d = rng.uniform(0,R0)
    φ = rng.uniform(0,2*math.pi)
    lat = clat + (d/111)*math.cos(φ)
    lon = clon + (d/111)*math.sin(φ)/math.cos(math.radians(lat0))
    if flags['ic_cg_mix']:
        flash_type = 'IC' if rng.rand()<0.25 else 'CG'
        n_strokes  = rng.randint(1,3) if flash_type=='IC' else rng.randint(1,6)
        amp_scale  = 0.4 if flash_type=='IC' else 1.0
    else:
        flash_type, n_strokes, amp_scale = 'CG', rng.randint(1,4), 1.0
    stroke_times=[t]
    for _ in range(n_strokes-1):
        stroke_times.append(stroke_times[-1]+rng.uniform(0.01,0.05))
    events.append(dict(id=eid, flash_type=flash_type, lat=lat, lon=lon,
                       stroke_times=stroke_times, amp_scale=amp_scale))

# ------------------------------------------------------------
# 7)  Build bursts and truth labels
# ------------------------------------------------------------
for ev in events:
    for si, t0 in enumerate(ev['stroke_times']):
        shape_tag = 'default'
        for nm in STN:
            dist = hav(ev['lat'],ev['lon'], stations[nm]['lat'],stations[nm]['lon'])
            jitter = rng.uniform(-50,50)/1e6 if flags['sprite_ring'] else 0
            arr_s  = t0 + dist/300_000 + jitter
            idx    = int(arr_s*FS)
            if idx>=N: continue
            # choose base waveform
            if flags['burst_diversity'] and rng.rand()<0.15:
                # simple K‑change shape
                tau=0.0007; burst = (np.arange(wave_len)/FS/tau *
                         np.exp(1-np.arange(wave_len)/FS/tau)).astype(np.float32)
                shape_tag='K'
            else:
                freq=rng.uniform(3000,9000)
                tau=rng.uniform(0.0004,0.002)
                burst = np.sin(2*math.pi*freq*tv_burst)*np.exp(-tv_burst/tau)
            # attenuation & amplitude
            amp=ev['amp_scale']*rng.uniform(2,4)/(1+dist/50)
            if flags['low_snr']: amp*=0.5
            burst=(amp*burst).astype(np.float32)
            # Q‑factor filtering (tier≥7)
            if flags['skywave'] and dist>400:
                f = np.fft.rfftfreq(wave_len,1/FS)
                H = np.exp(-0.00015*dist*((f/8000)**2))
                burst = np.fft.irfft(np.fft.rfft(burst)*H, n=wave_len).astype(np.float32)
            # multipath / skywave
            if flags['multipath'] and dist>60:
                dly = int(0.002*FS*min(1,dist/400))
                if dly<wave_len: burst[dly:]+=0.3*burst[:-dly]
            if flags['sprite_ring'] and rng.rand()<0.05:
                dly=int(rng.uniform(0.01,0.02)*FS)
                if dly<wave_len: burst[dly:]+=0.2*burst[:-dly]
            burst_book.append((nm, idx, burst))
            station_truth[nm][idx//1024]=True
            stroke_records.append(dict(event_id=ev['id'],stroke_i=si,
                                       station=nm,flash_type=ev['flash_type'],
                                       lat=ev['lat'],lon=ev['lon'],
                                       true_time_s=t0, sample_idx=idx,
                                       window_idx=idx//1024,
                                       shape=shape_tag))
            # earliest arrival sample for regression later
            if nm==STN[0]: stroke_samples.append(idx)

# ------------------------------------------------------------
# 8)  Noise configuration per station
# ------------------------------------------------------------
noise_col = {}
for nm in STN:
    if not flags['coloured_noise']:
        noise_col[nm]=dict(white=0.01, hum=0.01, tones=[], gain=1.0, skew=0.0)
        continue
    # coloured: random but deterministic by station hash
    r = random.Random(hash(nm)+SEED)
    tones=[]
    if flags['rfi_tones'] and r.random()<0.7:
        f=r.choice(rfi_freqs); amp=r.uniform(0.001,0.004)
        tones=[(f,amp)]
    noise_col[nm]=dict(
        white=r.uniform(0.01,0.022),
        hum=r.uniform(0.006,0.018),
        tones=tones,
        gain=1.0,
        skew=r.uniform(-15e-6,15e-6) if flags['clock_skew'] else 0.0
    )

# ------------------------------------------------------------
# 9)  Synthesis loop
# ------------------------------------------------------------
b,a = butter(4, 45000/(FS/2),'low')
chunk=int(20*FS)

for nm in STN:
    bur = [b for b in burst_book if b[0]==nm]
    cfg = noise_col[nm]
    g_drift = rng.uniform(-0.05,0.05) if flags['gain_drift'] else 0
    tvec_full = np.arange(N)/FS
    # possible drop‑out mask
    drop_mask = np.ones(N, bool)
    if flags['dropouts'] and rng.rand()<0.15:
        for _ in range(rng.randint(1,3)):
            s=int(rng.uniform(pre_sec*FS,N-0.5*FS)); e=s+int(0.3*FS)
            drop_mask[s:e]=False
    for s0 in range(0,N,chunk):
        e0=min(N,s0+chunk); L=e0-s0
        tvec=tvec_full[s0:e0]
        # base noise
        noise = cfg['white']*rng.randn(L) + cfg['hum']*np.sin(2*math.pi*50*tvec)
        for f,amp in cfg['tones']:
            noise += amp*np.sin(2*math.pi*f*tvec + rng.uniform(0,2*math.pi))
        seg=noise.copy()
        # gain drift
        seg *= (1+g_drift*(tvec-pre_sec)/(storm_sec+1e-9))
        # add bursts
        for (_,i0,br) in bur:
            if s0<=i0<e0:
                off=i0-s0; l=min(len(br),L-off)
                seg[off:off+l]+=br[:l]
        # false transients
        if flags['false_transient'] and rng.rand()<0.002:
            idx=rng.randint(0,L-wave_len)
            seg[idx:idx+wave_len]+=0.6*np.sin(2*math.pi*6000*tv_burst)*np.exp(-tv_burst/0.0008)
        # clipping
        if flags['clipping']:
            seg=np.clip(seg,-0.8*VREF,0.8*VREF)
        # low‑pass & ADC
        filt=filtfilt(b,a,seg)
        fullscale=2**(BITS-1)-1
        adc=np.clip(np.round(filt/VREF*fullscale), -fullscale, fullscale).astype(np.int16)
        quantized[nm][s0:e0][drop_mask[s0:e0]] = (adc[drop_mask[s0:e0]] *
                                                  (1+cfg['gain'])).astype(np.int16)

# ------------------------------------------------------------
# 10)  DataFrames
# ------------------------------------------------------------
df_wave=pd.DataFrame({'time_s':np.arange(N)/FS})
for nm in STN: df_wave[nm]=quantized[nm]
df_labels=pd.DataFrame(stroke_records)

# ------------------------------------------------------------
# 11)  Convenience converters
# ------------------------------------------------------------
def df_to_quantized(df): return {nm:df[nm].values.astype(np.int16) for nm in STN}
def df_labels_to_events(df):
    out=[]
    for eid,g in df.groupby('event_id'):
        out.append(dict(id=eid,flash_type=g.flash_type.iloc[0],
                        lat=g.lat.iloc[0],lon=g.lon.iloc[0],
                        stroke_times=sorted(g.true_time_s.unique())))
    return out

# ------------------------------------------------------------
# 12)  Summary printout
# ------------------------------------------------------------
print(f"Tier‑{DIFFICULTY}  |  cells={n_cells}  flashes={len(events)}  "
      f"strokes={len(df_labels)//len(STN)}  samples={N:,}")
print("df_wave:", df_wave.shape, " df_labels:", df_labels.shape)


In [ ]:
# ============================================================
# Cell 1 – Leela‑scope Lightning Simulator v3  (tiers 1 … 9)
# ============================================================
"""
Exactly six stations, harder ‘near/medium/far’, and progressive realism.

Variables & structures are IDENTICAL to the legacy generator:
    stations, station_order, STN           – geography dict / list / alias
    STATIONS                               – extra alias for downstream code
    quantized, station_truth               – raw ADC & truth windows
    events, stroke_records, stroke_samples – ground‑truth meta
    df_wave, df_labels                     – convenience DataFrames
    df_to_quantized(),  df_labels_to_events()
Nothing else downstream needs to change.
"""
# ------------------------------------------------------------
import math, random, numpy as np, pandas as pd
from scipy.signal import butter, filtfilt
# 0)  USER KNOBS ------------------------------------------------------------
SEED          = 424242
duration_min  = 5                 # storm length (min)
scenario      = 'far'          # 'near' | 'medium' | 'far'
DIFFICULTY    = 9                 # 1 (easy) … 9 (very hard)
FS            = 109_375           # Hz  – keep for pipeline
BITS, VREF    = 14, 1.0
random.seed(SEED); np.random.seed(SEED)
rng = np.random.default_rng(SEED)

# 1)  STATION GEOGRAPHY (always six) ---------------------------------------
stations = {
    'LER': dict(lat=60.15, lon=-1.13),   # Lerwick
    'INV': dict(lat=57.48, lon=-4.23),   # Inverness
    'DUB': dict(lat=53.35, lon=-6.26),   # Dublin
    'LON': dict(lat=51.50, lon=-0.12),   # London
    'AMS': dict(lat=52.37, lon= 4.90),   # Amsterdam
    'BER': dict(lat=52.52, lon=13.40),   # Berlin
}
station_order = list(stations.keys())    # fixed order
STN      = station_order
STATIONS = station_order                 # alias used in other notebooks
N_STN    = len(STN)

# 2)  Helpers ---------------------------------------------------------------
def hav(lat1, lon1, lat2, lon2):
    R=6371.0
    φ1,φ2 = map(math.radians, (lat1,lat2))
    dφ    = math.radians(lat2-lat1)
    dλ    = math.radians(lon2-lon1)
    a = math.sin(dφ/2)**2 + math.cos(φ1)*math.cos(φ2)*math.sin(dλ/2)**2
    return 2*R*math.asin(math.sqrt(a))

# 3)  Timeline --------------------------------------------------------------
pre_sec   = rng.uniform(5,30)
storm_sec = duration_min*60
total_sec = pre_sec + storm_sec
N         = int(total_sec*FS)

quantized      = {nm: np.zeros(N, np.int16)   for nm in STN}
station_truth  = {nm: np.zeros(N//1024, bool) for nm in STN}
events, stroke_records, burst_book, stroke_samples = [], [], [], []

# 4)  Tier flags ------------------------------------------------------------
flags = dict(
    ic_mix          = DIFFICULTY>=2,
    multipath       = DIFFICULTY>=3,
    coloured_noise  = DIFFICULTY>=4,
    rfi             = DIFFICULTY>=5,
    sprite_ring     = DIFFICULTY>=5,
    false_transient = DIFFICULTY>=5,
    clipping        = DIFFICULTY>=5,
    multi_cell      = DIFFICULTY>=6,
    skywave         = DIFFICULTY>=7,
    bg_sferics      = DIFFICULTY>=7,
    clock_skew      = DIFFICULTY>=8,
    gain_drift      = DIFFICULTY>=8,
    dropouts        = DIFFICULTY>=8,
    low_snr         = DIFFICULTY>=9,
    burst_div       = DIFFICULTY>=9,
)

# 5)  Storm‑cell geometry ----------------------------------------------------
lat0 = np.mean([s['lat'] for s in stations.values()])
lon0 = np.mean([s['lon'] for s in stations.values()])
R0   = dict(near=100, medium=400, far=900)[scenario]   # tougher radii

n_cells = 1 if not flags['multi_cell'] else rng.integers(2,5)
cells=[]
for _ in range(n_cells):
    θ   = rng.uniform(0,2*math.pi)
    rad = R0*rng.uniform(0.3,1.0)
    cells.append(dict(
        lat   = lat0 + (rad/111)*math.cos(θ),
        lon   = lon0 + (rad/111)*math.sin(θ)/math.cos(math.radians(lat0)),
        drift = rng.uniform(-0.20,0.20,2)          # deg·h‑1
    ))

# 6)  Flash & stroke generation ---------------------------------------------
wave_len = int(0.04*FS)
tv       = np.arange(wave_len)/FS
rfi_freqs= [14400,20100,30300]

eid, t = 0, pre_sec
while True:
    cell = cells[rng.integers(len(cells))]
    c_age= t-pre_sec
    c_lat= cell['lat'] + cell['drift'][0]*c_age/3600
    c_lon= cell['lon'] + cell['drift'][1]*c_age/3600
    t   += rng.lognormal(3,1)* (0.4 if flags['multi_cell'] else 1)
    if t>=total_sec: break
    eid += 1
    # flash location
    d = rng.uniform(0,R0)
    φ = rng.uniform(0,2*math.pi)
    f_lat = c_lat + (d/111)*math.cos(φ)
    f_lon = c_lon + (d/111)*math.sin(φ)/math.cos(math.radians(lat0))
    f_type= 'IC' if (flags['ic_mix'] and rng.random()<0.3) else 'CG'
    n_str = rng.integers(1, 4 if f_type=='IC' else 7)
    amp0  = 0.35 if f_type=='IC' else 1.0
    s_times = sorted(t + rng.uniform(0,0.06,size=n_str))
    events.append(dict(id=eid,flash_type=f_type,lat=f_lat,lon=f_lon,
                       stroke_times=s_times))
    # build bursts
    for si, t0 in enumerate(s_times):
        for nm in STN:
            dist = hav(f_lat,f_lon, stations[nm]['lat'],stations[nm]['lon'])
            idx  = int((t0 + dist/300_000 + rng.uniform(-50,50)/1e6)*FS)
            if idx>=N: continue
            # waveform
            if flags['burst_div'] and rng.random()<0.15:
                tau=.0008; burst = (tv/tau)*np.exp(1-tv/tau)
            else:
                f0=rng.uniform(2500,9500); tau=rng.uniform(0.0003,0.0025)
                burst = np.sin(2*math.pi*f0*tv)*np.exp(-tv/tau)
            amp = amp0*rng.uniform(2,5)/(1+dist/40)
            if flags['low_snr']: amp*=0.4
            burst *= amp
            # multipath
            if flags['multipath'] and dist>60:
                dly=int(rng.uniform(0.001,0.0035)*FS)
                if dly<wave_len: burst[dly:]+=0.35*burst[:-dly]
            # sprite ringers
            if flags['sprite_ring'] and rng.random()<0.04:
                dly=int(rng.uniform(0.008,0.018)*FS)
                if dly<wave_len: burst[dly:]+=0.25*burst[:-dly]
            # sky‑wave attenuation
            if flags['skywave'] and dist>600:
                f=np.fft.rfftfreq(wave_len,1/FS)
                H=np.exp(-0.00025*dist*((f/6e3)**2))
                burst=np.fft.irfft(np.fft.rfft(burst)*H,n=wave_len)
            burst_book.append((nm,idx,burst.astype(np.float32)))
            station_truth[nm][idx//1024]=True
            stroke_records.append(dict(event_id=eid,stroke_i=si,station=nm,
                                       flash_type=f_type,lat=f_lat,lon=f_lon,
                                       true_time_s=t0,sample_idx=idx,
                                       window_idx=idx//1024))
            if nm==STN[0]: stroke_samples.append(idx)

# 7)  Noise profiles ---------------------------------------------------------
noise_cfg={}
for nm in STN:
    base_white=rng.uniform(0.010,0.017)
    tones=[]
    if flags['rfi'] and rng.random()<0.6:
        tones=[(rng.choice(rfi_freqs), rng.uniform(0.001,0.004))]
    noise_cfg[nm]=dict(
        w = base_white if not flags['coloured_noise'] else base_white*rng.uniform(1,1.8),
        h = 0.01 if not flags['coloured_noise'] else rng.uniform(0.006,0.020),
        tones = tones,
        gain_drift = rng.uniform(-0.05,0.05) if flags['gain_drift'] else 0.0,
        skew = rng.uniform(-20e-6,20e-6) if flags['clock_skew'] else 0.0
    )

# 8)  ADC synthesis loop -----------------------------------------------------
b,a  = butter(4, 45000/(FS/2),'low')
chunk= int(20*FS)
tv40 = np.arange(wave_len)/FS   # for false‑transient reuse

for nm in STN:
    bur = [b for b in burst_book if b[0]==nm]
    cfg = noise_cfg[nm]
    drop = np.ones(N,bool)
    if flags['dropouts'] and rng.random()<0.1:
        for _ in range(rng.integers(1,3)):
            s=rng.integers(int(pre_sec*FS),N-int(0.4*FS))
            drop[s:s+int(0.4*FS)]=False
    for s0 in range(0,N,chunk):
        e0=min(N,s0+chunk); L=e0-s0
        t=np.arange(s0,e0)/FS
        seg = cfg['w']*rng.standard_normal(L) + cfg['h']*np.sin(2*math.pi*50*t)
        for f,amp in cfg['tones']:
            seg += amp*np.sin(2*math.pi*f*t + rng.uniform(0,2*math.pi))
        # gain drift
        seg *= 1 + cfg['gain_drift']*(t-pre_sec)/(storm_sec+1e-9)
        # add bursts
        for (_,i0,br) in bur:
            if s0<=i0<e0:
                off=i0-s0; l=min(len(br),L-off)
                seg[off:off+l]+=br[:l]
        # false transient
        if flags['false_transient'] and rng.random()<0.002:
            idx=rng.integers(0,L-wave_len)
            seg[idx:idx+wave_len]+=0.7*np.sin(2*math.pi*5800*tv40)*np.exp(-tv40/0.0009)
        # filtering & clipping
        seg = filtfilt(b,a,seg)
        if flags['clipping']: seg=np.clip(seg,-0.9*VREF,0.9*VREF)
        full=2**(BITS-1)-1
        adc = np.clip(np.round(seg/VREF*full), -full, full).astype(np.int16)
        quantized[nm][s0:e0][drop[s0:e0]] = adc[drop[s0:e0]]

# 9)  DataFrames -------------------------------------------------------------
df_wave = pd.DataFrame({'time_s':np.arange(N)/FS})
for nm in STN: df_wave[nm]=quantized[nm]
df_labels = pd.DataFrame(stroke_records)

def df_to_quantized(df):      return {nm:df[nm].values.astype(np.int16) for nm in STN}
def df_labels_to_events(df):
    out=[];      grp=df.groupby('event_id')
    for eid,g in grp:
        out.append(dict(id=eid,flash_type=g.flash_type.iloc[0],
                        lat=g.lat.iloc[0],lon=g.lon.iloc[0],
                        stroke_times=sorted(g.true_time_s.unique())))
    return out

# 10)  Summary --------------------------------------------------------------
print(f"Tier‑{DIFFICULTY} | scenario={scenario} | cells={n_cells}")
print(f"Flashes {len(events):3d} | strokes {len(df_labels)//N_STN:3d} "
      f"| samples {N:,}")


In [ ]:
df_labels

In [ ]:
# ============================================================
# Cell 1‑QA‑PLUS  –  Rich “eyes‑on” dashboard for the simulator
# ============================================================
#  • Requires matplotlib, pandas, scipy                                                │
#  • Optional: cartopy  (for the Europe station map)                                   │
#  • ZERO side‑effects – everything is read‑only, later cells still work unmodified.   │
# ============================================================

import numpy as np, pandas as pd, matplotlib.pyplot as plt
from scipy.signal import hilbert
plt.rcParams.update({'axes.grid': True, 'figure.dpi': 110})

# -----------------------------------------------------------------
# 0)  Helper: concise console table printer
# -----------------------------------------------------------------
def _tbl(rows, hdr=None, col_sep="  "):
    if hdr: print(col_sep.join(hdr))
    for r in rows:
        print(col_sep.join(str(c) for c in r))
    print()

# -----------------------------------------------------------------
# 1)  Global metadata
# -----------------------------------------------------------------
N   = quantized[station_order[0]].size
dur = N / FS
print(f"► Simulator difficulty tier : {DIFFICULTY}")
print(f"► Sampling rate             : {FS:,.0f} Hz "
      f"(Δt={1e6/FS:.2f} µs)")
print(f"► Duration                  : {dur:.2f} s  ({dur/60:.2f} min)")
print(f"► Total ADC samples         : {N:,}\n")

# -----------------------------------------------------------------
# 2)  Per‑station ADC statistics
# -----------------------------------------------------------------
rows=[]
for nm in station_order:
    q = quantized[nm].astype(float)
    rows.append([nm, q.min().astype(int), q.max().astype(int),
                 f"{q.mean():.1f}", f"{q.std():.1f}",
                 f"{100*np.count_nonzero(q)/len(q):.2f}%"])
_tbl(rows, hdr=["STN","min","max","μ","σ","non‑zero %"])

# -----------------------------------------------------------------
# 3)  Flash / stroke timing & location dataframe
# -----------------------------------------------------------------
stroke_times = np.hstack([ev["stroke_times"] for ev in events])
lat_rep      = np.hstack([[ev['lat']]*len(ev['stroke_times']) for ev in events])
lon_rep      = np.hstack([[ev['lon']]*len(ev['stroke_times']) for ev in events])
df_strokes   = pd.DataFrame(dict(time_s=stroke_times,
                                 lat=lat_rep, lon=lon_rep))
df_strokes.sort_values("time_s", inplace=True, ignore_index=True)
print(f"Flashes  : {len(events)}")
print(f"Strokes  : {len(df_strokes)}\n")
display(df_strokes.head())

# -----------------------------------------------------------------
# 4)  Simple “did station see the stroke?” heuristic
# -----------------------------------------------------------------
pre_samp = int(pre_sec*FS)                       # noise estimate window
det_tbl  = []
for nm in station_order:
    noiseσ = quantized[nm][:pre_samp].astype(float).std()
    thr    = 3*noiseσ
    hits   = []
    for t in df_strokes.time_s:
        idx = int(t*FS)
        hits.append(abs(quantized[nm][idx])>=thr if idx<N else False)
    df_strokes[f"det_{nm}"] = hits
    det_tbl.append([nm, f"{100*np.mean(hits):.1f}%"])

_tbl(det_tbl, hdr=["STN","simple detection rate"])

# -----------------------------------------------------------------
# 5)  Window‑level labels (quiet = 0, lightning = 1)
# -----------------------------------------------------------------
W      = 1024
n_win  = N // W
starts = (np.arange(n_win)*W)/FS
stroke_idx = (df_strokes.time_s.values*FS).astype(int)
labels_win = np.zeros(n_win, bool)
labels_win[(stroke_idx//W)] = True           # even if >1 stroke per win → still 1

df_win = pd.DataFrame(dict(win_idx=np.arange(n_win, dtype=int),
                           start_s=starts,
                           label=labels_win.astype(int)))
quiet_cnt, light_cnt = np.bincount(df_win.label, minlength=2)
print(f"\nWindows: quiet={quiet_cnt}, lightning={light_cnt}")

# -----------------------------------------------------------------
# 6)  Visual #1 – histograms (log y)
# -----------------------------------------------------------------
for nm in station_order:
    plt.figure(figsize=(4,2.5))
    plt.hist(quantized[nm][::max(1,N//200_000)],
             bins=120, log=True, color='#4682B4')
    plt.title(f"{nm} – ADC histogram")
    plt.xlabel("ADC counts"); plt.ylabel("occurrences (log)")
    plt.tight_layout(); plt.show()

# -----------------------------------------------------------------
# 7)  Visual #2 – first lightning vs first quiet window
# -----------------------------------------------------------------
def _plot_window(win_row, title, colour):
    idx = int(win_row.win_idx)
    tms = (np.arange(W)/FS)*1e3
    plt.figure(figsize=(6,2.4))
    for nm in station_order:
        seg = quantized[nm][idx*W:(idx+1)*W]
        plt.plot(tms, seg, label=nm, alpha=.8)
    plt.title(title); plt.xlabel("time (ms)"); plt.ylabel("ADC")
    plt.legend(ncol=len(station_order)//2)
    plt.tight_layout(); plt.show()

_plot_window(df_win[df_win.label==1].iloc[0],
             "First lightning window", 'orange')
_plot_window(df_win[df_win.label==0].iloc[0],
             "First quiet window", 'gray')

# -----------------------------------------------------------------
# 8)  Visual #3 – analytic‑signal envelope of lightning window
# -----------------------------------------------------------------
idx_lit = int(df_win[df_win.label==1].iloc[0].win_idx)
tms     = (np.arange(W)/FS)*1e3
plt.figure(figsize=(6,2.6))
for nm in station_order:
    seg = quantized[nm][idx_lit*W:(idx_lit+1)*W].astype(float)
    env = np.abs(hilbert(seg))
    plt.plot(tms, env, label=f"{nm} env")
plt.title(f"Envelope – window {idx_lit} (lightning)")
plt.xlabel("time (ms)"); plt.ylabel("abs(hilbert)")
plt.legend(ncol=len(station_order)//2); plt.tight_layout(); plt.show()

# -----------------------------------------------------------------
# 9)  Visual #4 – stroke‑window heat‑map (sparse but visible)
# -----------------------------------------------------------------
truth_mat = np.vstack([station_truth[nm][:n_win] for nm in station_order])
plt.figure(figsize=(10,1.2+0.25*len(station_order)))
plt.imshow(truth_mat, aspect='auto',
           cmap=plt.get_cmap("Reds", 2), interpolation='nearest')
plt.yticks(range(len(station_order)), station_order)
plt.xlabel(f"window index ({W} samples)"); plt.title("Ground‑truth stroke windows")
# over‑plot thicker bars for visibility
for w in np.where(truth_mat.any(0))[0]:
    plt.axvline(w, color='red', lw=.5, alpha=.3)
plt.tight_layout(); plt.show()

# -----------------------------------------------------------------
# 10) Optional geographic snapshot (requires Cartopy)
# -----------------------------------------------------------------
try:
    import cartopy.crs as ccrs, cartopy.feature as cfeature
    fig = plt.figure(figsize=(6,4))
    ax  = plt.axes(projection=ccrs.PlateCarree())
    ax.add_feature(cfeature.COASTLINE, lw=.6); ax.add_feature(cfeature.BORDERS,lw=.4)
    ax.set_extent([-35, 30, 30, 65])
    ax.scatter(df_strokes.lon, df_strokes.lat, c='orange', s=10, lw=0, zorder=2)
    for nm in station_order:
        ax.plot(stations[nm]['lon'], stations[nm]['lat'], '^', ms=7, mfc='#1f77b4',
                mec='k', zorder=3)
        ax.text(stations[nm]['lon']+0.4, stations[nm]['lat']+0.4, nm, fontsize=8)
    ax.set_title("Station & flash geography"); plt.tight_layout(); plt.show()
except ImportError:
    print("(Cartopy not installed – map skipped)\n")


In [ ]:
# ============================================================
# Cell 2 – Robust stroke‐by‐stroke Hilbert‐envelope detector
#            + 50%‐overlap windows + percentile thresholds
#            (earliest‐arrival indexing)
# ============================================================

"""
Cell 2 – Robust Stroke‐by‐Stroke Hilbert‐Envelope Detector
-----------------------------------------------------------

This cell implements a fully‐unsupervised lightning‐stroke detector based on
the analytic‐signal envelope (Hilbert transform) of each station’s ADC trace,
using overlapping windows and percentile thresholding to flag “hot” windows
that likely contain sferic bursts.  It then fuses detections across stations
to produce both station‐level and network‐level stroke metrics.

Key Design & Configuration
--------------------------
FS          : Sampling rate (Hz), inherited from Cell 1.
WIN         : Window length in samples (1024), ~9.4 ms at 109 375 Hz.
HOP         : Hop size (WIN//2 = 512), 50 % overlap to ensure each stroke
              will straddle at least one window in its half‐width.
PCT_THRESH  : Percentile threshold (99.9), marking the top 0.1 % of all
              window‐peak envelopes as candidate sferics.
MIN_STN     : Network‐level requirement—only count an event if ≥2 stations
              independently flag a hot window.
TOL_WIN     : ±1‐window tolerance around the theoretical arrival index,
              accommodating small timing or rounding variations.

1) Compute Hilbert‐Envelope Window Peaks
   ────────────────────────────────────────
   - For each station’s full‐length integer signal, compute the analytic
     signal envelope via `abs(hilbert(raw))`.
   - Slide a WIN‐sample window with HOP stride, recording the maximum
     envelope amplitude per window.  This yields a per‐station peaks array
     of length n_win = ⌊(N−WIN)/HOP⌋+1.

2) Threshold Hot Windows
   ───────────────────────
   - For each station, compute the PCT_THRESH‐th percentile of its first
     n_win peaks.  This percentile is data‐driven, requiring no prior
     quiet period.
   - Flag windows whose peak envelope exceeds this threshold as “hot”.
   - Print per‐station thresholds and counts for audit.

3) Build Stroke Sample Indices (“Ground Truth”)
   ─────────────────────────────────────────────
   - From the simulator’s `events` list, gather every true stroke time t0.
   - For each station, compute the propagation delay = distance/300 000 km/s,
     translate to sample index i₀ = int((t0 + delay)·FS), and record all
     arrivals.
   - Use the **earliest** arrival among stations as the reference index
     for that stroke.  This ties each true stroke to one canonical sample.

4) Count Station Hits per Stroke
   ─────────────────────────────────
   - For each recorded stroke sample i₀, convert to a central window index
     w = i₀//HOP.
   - Check the ±TOL_WIN window band [w−1 … w+1] in each station’s hot‐mask.
   - Increment the stroke’s station‐hit count if any hot window overlaps.

5) Station‐Level Metrics
   ───────────────────────
   - For each station, build a binary hit vector: did station n flag any
     hot window in the tolerance band around each i₀?
   - Compute confusion matrix (TP, FP, FN, TN) and derive Precision, Recall,
     F₁‐score against the all‐true stroke list.
   - This measures how well each individual sensor alone would perform.

6) Network‐Level Metrics
   ───────────────────────
   - Define network detection if ≥MIN_STN stations flagged the event.
   - Compute overall TP, FP, FN, TN and network‐level P, R, F₁.

Strengths
---------
- **Unsupervised**: threshold derives from data‐distribution per station.
- **Overlap**: 50 % window overlap ensures robustness to stroke timing.
- **Interpretable**: envelope peaks directly map to burst energy.
- **Station‐fusion**: multi‐sensor voting reduces false alarms.

Weaknesses & Possible Improvements
----------------------------------
- **Fixed window** may not match variable sferic durations; consider
  adaptive or waveform‐matched windows.
- **Global percentile** can drift if storm‐activity proportion changes;
  a rolling or pre‐storm baseline might stabilise thresholds.
- **No frequency‐domain tuning**: could augment with band‐limited energy
  or wavelet‐based features to improve SNR in noise‐coloured settings.
- **No cross‐correlation**: arrival‐time estimation could be refined by
  cross‐correlating station pairs for sub‐sample precision.
- **No amplitude normalisation**: stations at different distances see
  systematically smaller envelopes; a distance‐aware gain correction
  could equalise detection sensitivity.

Usage
-----
Run this cell after Cell 1’s simulation.  It prints station‐wise thresholds,
counts, and detection metrics, enabling rapid evaluation of classical
envelope‐based detection before progressing to advanced ML or compression
methods.
"""


import numpy as np
from scipy.signal import hilbert
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from collections import Counter

# ── Parameters ──────────────────────────────────────────────
FS         = float(FS)         # sampling rate from Cell 1
WIN        = 1024              # window size
HOP        = WIN // 2          # 50% overlap hop
STATIONS   = station_order     # ['LON','LER','DUB','BER']
PCT_THRESH = 99.9              # top 0.1% of envelope peaks
MIN_STN    = 2                 # require ≥2 stations → stroke detection
TOL_WIN    = 1                 # ±1‐window tolerance for scoring

# ── 1) Compute Hilbert‐envelope window peaks per station ────
def window_peaks(raw):
    env   = np.abs(hilbert(raw.astype(float)))
    n_win = (len(env) - WIN)//HOP + 1
    peaks = np.empty(n_win, float)
    for i in range(n_win):
        s = i * HOP
        peaks[i] = env[s:s+WIN].max()
    return peaks

peaks = {nm: window_peaks(quantized[nm]) for nm in STATIONS}
n_win = min(len(v) for v in peaks.values())

# ── 2) Threshold by percentile of all windows ───────────────
hot = {}
print("Per-station thresholds & flagged windows:")
for nm in STATIONS:
    p    = peaks[nm][:n_win]
    thr  = np.percentile(p, PCT_THRESH)
    mask = p > thr
    hot[nm] = mask
    print(f" {nm}: thr={thr:6.1f}, flagged={mask.sum():5d} / {n_win}")

# ── 3) Build stroke_samples using earliest arrival across stations ─
stroke_samples = []
for ev in events:
    for t0 in ev['stroke_times']:
        arrivals = []
        for nm in STATIONS:
            geo  = stations[nm]
            dist = hav(ev['lat'], ev['lon'], geo['lat'], geo['lon'])
            i0   = int((t0 + dist/300_000.0) * FS)
            arrivals.append(i0)
        stroke_samples.append(min(arrivals))
stroke_samples = np.array(stroke_samples)
n_events      = len(stroke_samples)
stroke_truth  = np.ones(n_events, bool)

# ── 4) Count station hits per stroke (within ±TOL_WIN windows) ─
counts = np.zeros(n_events, int)
for nm in STATIONS:
    mask = hot[nm]
    for j, i0 in enumerate(stroke_samples):
        w = i0 // HOP
        w0 = max(0, w - TOL_WIN)
        w1 = min(n_win, w + TOL_WIN + 1)
        if mask[w0:w1].any():
            counts[j] += 1

print("\nStations ≥thr per stroke:")
for k, v in sorted(Counter(counts).items()):
    print(f"  {k:2d} stations → {v} strokes")

# ── 5) Station-level stroke metrics ───────────────────────────
print("\nStation-level stroke detection:")
print("stn   TP   FP   FN     P     R    F1")
for nm in STATIONS:
    hits = np.array([
        hot[nm][max(0, min(n_win-1, i0//HOP))]
        for i0 in stroke_samples
    ])
    tn, fp, fn, tp = confusion_matrix(stroke_truth, hits, labels=[False,True]).ravel()
    P = precision_score(stroke_truth, hits, zero_division=0)
    R = recall_score   (stroke_truth, hits, zero_division=0)
    F = f1_score       (stroke_truth, hits, zero_division=0)
    print(f"{nm:>3} {tp:4d} {fp:4d} {fn:4d} {P:8.3f} {R:6.3f} {F:6.3f}")

# ── 6) Network-level stroke metrics (≥MIN_STN stations) ──────
stroke_pred = counts >= MIN_STN
tn, fp, fn, tp = confusion_matrix(
    stroke_truth, stroke_pred, labels=[False,True]
).ravel()
P_net = precision_score(stroke_truth, stroke_pred, zero_division=0)
R_net = recall_score   (stroke_truth, stroke_pred, zero_division=0)
F_net = f1_score       (stroke_truth, stroke_pred, zero_division=0)

print(f"\nNetwork (≥{MIN_STN} stn) stroke-wise:")
print(f" TP={tp}  FP={fp}  FN={fn}  TN={tn}")
print(f" P={P_net:.3f}  R={R_net:.3f}  F1={F_net:.3f}")


In [ ]:
# ============================================================
# Cell 3 – Compact stroke table + time/location regressions
# ============================================================

"""
Cell 3 – Time‐of‐Arrival (TOA) Localization & Regression Analysis
------------------------------------------------------------------

This cell takes the “hot” windows identified in Cell 2 for each station,
extracts precise per‐station arrival times via the analytic‐signal envelope,
and then performs a robust least‐squares multilateration to estimate the
origin time and geographic location of each lightning stroke.  Finally, it
builds a comprehensive summary table and produces regression diagnostics
for both time and distance estimates.

Key Configuration & Constants
-----------------------------
FS       : Sampling rate (Hz), carried over from Cell 1.
WIN, HOP : Window length (1024 samples) and hop (512 samples) as in Cell 2.
STATIONS : List of station IDs under analysis.
c        : Speed of light (km/s) for propagation delays.
Re       : Earth radius (km) for geodesy.

1) Coordinate Transforms & Distance Utilities
   ───────────────────────────────────────────
   - `ll_xyz(lat, lon)`: Converts (latitude, longitude) → 3D ECEF coordinates.
   - `hav_km(...)`: Haversine formula yielding great‐circle distance (km).
   - Precompute each station’s XYZ once for efficient TOA matrix builds.

2) Precise Arrival‐Time Picking
   ─────────────────────────────
   - For each station that flagged a “hot” window for a given stroke sample s0:
     • Extract a ±WIN‐sample segment around s0.
     • Compute the analytic‐signal envelope via `abs(hilbert(segment))`.
     • Find the envelope peak index, convert back to global sample index,
       and then to time (s).  This yields sub‐millisecond TOA estimates.

3) Multilateration via Robust Least‐Squares
   ────────────────────────────────────────
   - Only attempt localization if ≥3 stations have valid TOAs.
   - Initial guess:
       • Take the mean position of the earliest‐arriving three stations.
       • Estimate origin time T0 by back‐projecting first TOA.
   - Iterative Gauss–Newton:
       • Residual = observed TOAs – (T0 + distance/c).
       • Jacobian J = ∂residual/∂(x,y,z,T0) constructed from station positions.
       • Solve linearised normal equations to update (x,y,z,T0).
       • Stop when residual RMS < 0.2 ms or after 8 iterations.
   - Sanity checks:
       • Reject if any single update suggests >500 km jump.
       • Reject if final estimate lies >800 km from network centroid.
   - Output: (T0_est, lat_est, lon_est) or None if no valid solution.

4) Truth Arrays & DataFrame Assembly
   ─────────────────────────────────
   - Extract true stroke times and locations from `events`.
   - For each stroke k:
       • Attempt `solve_toa` on its arrival dictionary.
       • If successful, record true vs. estimated T0, time error (ms),
         true/estimated distances to London, full loc error (km),
         number of stations used, and per‐station estimated distances.
       • If unsuccessful, fill with NaNs.

5) Regression Diagnostics & Plots
   ───────────────────────────────
   - **Time Regression**: Fit T_est = m·T_true + b, compute R².
   - **Distance Regression**: Fit D_est(London) = m·D_true(London) + b, R².
   - **Histograms**: Distribution of localization errors (km).
   - **Scatter+Fits**: Overlay regression lines on scatter of true vs.
     estimated time and distance, with slopes & R² annotated.

Strengths
---------
  • **Precision**: Envelope‐based arrival picking yields sub‐ms TOAs.
  • **Robustness**: Iterative Gauss–Newton with outlier checks guards
    against poor initial guesses and non‐linearities.
  • **Comprehensive**: Outputs full table of per‐stroke metrics plus
    station‐wise distance breakdown.

Weaknesses & Caveats
--------------------
  • **Assumed Straight‐Line Propagation**: Ignores ionospheric or
    terrain‐induced multipath delays (could bias remote events).
  • **Linearisation**: May fail for widely spaced stations if initial
    guess is poor; more sophisticated global optimisers (e.g. LM,
    MCMC) could improve convergence.
  • **Rejection Criteria**: Hard thresholds (500 km, 800 km) may need
    tuning for real‐world geometries or extended arrays.
  • **Station Geometry Dependence**: Accuracy varies strongly with
    station GDOP; sparse or colinear arrays degrade position precision.

Usage
-----
Run immediately after Cell 2’s envelope‐based detector.  Review the
printed DataFrame (first 12 rows) for direct comparison of truth vs. estimate,
and examine the three diagnostic plots to judge overall time and spatial
accuracy of the multilateration algorithm.
"""


import numpy as np, pandas as pd, matplotlib.pyplot as plt
from scipy.signal import hilbert

plt.rcParams["font.family"] = "DejaVu Sans"
c, Re = 299_792.458, 6371.0            # km s‑1, Earth radius

# ---------- helpers ---------------------------------------------------------
def ll_xyz(lat, lon):
    lat, lon = np.radians(lat), np.radians(lon)
    cl = np.cos(lat)
    return np.array([Re*cl*np.cos(lon),
                     Re*cl*np.sin(lon),
                     Re*np.sin(lat)])

def hav_km(la1, lo1, la2, lo2):
    φ1, φ2 = map(np.radians, (la1, la2))
    dφ = φ2-φ1; dλ = np.radians(lo2-lo1)
    a = np.sin(dφ/2)**2 + np.cos(φ1)*np.cos(φ2)*np.sin(dλ/2)**2
    return 2*Re*np.arcsin(np.sqrt(a))

st_xyz = {nm: ll_xyz(stations[nm]['lat'], stations[nm]['lon']) for nm in STATIONS}

def pick_arrival(nm, s0):
    seg = quantized[nm][max(0,s0-WIN):min(len(quantized[nm]),s0+WIN)].astype(float)
    env = np.abs(hilbert(seg)); idx = env.argmax()
    return (max(0,s0-WIN)+idx)/FS

# ---------- build arrival dictionary from Cell‑2 results --------------------
arrivals = []
for j,s0 in enumerate(stroke_samples):
    d={}
    for nm in STATIONS:
        w=s0//HOP
        if w<n_win and hot[nm][w]:
            d[nm]=pick_arrival(nm,int(s0))
    arrivals.append(d)

# ---------- TOA least‑squares (robust) -------------------------------------
def solve_toa(a):
    if len(a)<3: return None
    st = list(a); t = np.array([a[n] for n in st]); S=np.stack([st_xyz[n] for n in st])
    r = S[np.argsort(t)[:3]].mean(0); T0 = t.min() - np.linalg.norm(S[np.argmin(t)]-r)/c
    for _ in range(8):
        d = np.linalg.norm(S-r,axis=1); d[d==0]+=1e-6
        res = t-(T0+d/c)
        if np.sqrt((res**2).mean())*1e3<0.2: break
        J=np.empty((len(st),4)); J[:,:3]=-(S-r)/(c*d)[:,None]; J[:,3]=-1
        delta,*_=np.linalg.lstsq(J,res,rcond=None);
        if np.linalg.norm(delta[:3])>500: return None
        r+=delta[:3]; T0+=delta[3]
    la=np.degrees(np.arcsin(r[2]/np.linalg.norm(r)))
    lo=np.degrees(np.arctan2(r[1],r[0]))
    if hav_km(la,lo,lat0,lon0)>800: return None
    return T0,la,lo

# ---------- truth vectors ---------------------------------------------------
tru_t, tru_la, tru_lo = [],[],[]
for ev in events:
    for t0 in ev['stroke_times']:
        tru_t.append(t0); tru_la.append(ev['lat']); tru_lo.append(ev['lon'])
tru_t, tru_la, tru_lo = map(np.array, (tru_t, tru_la, tru_lo))

# ---------- build neat table ------------------------------------------------
records=[]
for k,(t_true,la_true,lo_true) in enumerate(zip(tru_t,tru_la,tru_lo)):
    sol=solve_toa(arrivals[k])
    row={'#':k,'t_true':t_true}
    if sol:
        T0,la,lo = sol
        row.update(t_est=T0,
                   err_ms=(T0-t_true)*1e3,
                   dLON_true=hav_km(la_true,lo_true,
                                    stations['LON']['lat'],stations['LON']['lon']),
                   dLON_est =hav_km(la,lo,
                                    stations['LON']['lat'],stations['LON']['lon']),
                   loc_err_km=hav_km(la,lo,la_true,lo_true),
                   n_stn=len(arrivals[k]))
        for nm in STATIONS:
            row[f'd_{nm}']=hav_km(la,lo,stations[nm]['lat'],stations[nm]['lon']) \
                           if nm in arrivals[k] else np.nan
    else:
        row.update({col:np.nan for col in
            ['t_est','err_ms','dLON_true','dLON_est','loc_err_km','n_stn']+[f'd_{n}'for n in STATIONS]})
    records.append(row)

df=pd.DataFrame(records)
# Formatting
pd.set_option('display.max_columns', None)
show_cols=['#','t_true','t_est','err_ms','loc_err_km','dLON_true','dLON_est','n_stn']+[f'd_{n}' for n in STATIONS]
print("\nStroke table (first 12 rows):")
print(df[show_cols].head(12).round(3).fillna('').to_string(index=False))

# ---------- regressions & plots --------------------------------------------
mask=df.t_est.notna()
# time regression
coef=np.polyfit(df.loc[mask,'t_true'],df.loc[mask,'t_est'],1);
R2 = np.corrcoef(df.loc[mask,'t_true'],df.loc[mask,'t_est'])[0,1]**2

# distance‑to‑London regression
maskL=df.dLON_est.notna()
coefL=np.polyfit(df.loc[maskL,'dLON_true'],df.loc[maskL,'dLON_est'],1)
R2L = np.corrcoef(df.loc[maskL,'dLON_true'],df.loc[maskL,'dLON_est'])[0,1]**2

# plots
fig,(ax1,ax2,ax3)=plt.subplots(3,1,figsize=(6,10))

ax1.hist(df.loc[mask,'loc_err_km'],bins=10,edgecolor='k',alpha=.7)
ax1.set_title("Location error histogram"); ax1.set_xlabel("Error (km)"); ax1.set_ylabel("Count")

# time regression
ax2.scatter(df.loc[mask,'t_true'],df.loc[mask,'t_est'],s=25,alpha=.8)
x=np.linspace(df.t_true.min(),df.t_true.max(),100)
ax2.plot(x,coef[0]*x+coef[1],'r--')
ax2.set_title(f"Origin‑time fit  (slope={coef[0]:.6f}, R²={R2:.4f})")
ax2.set_xlabel("True T₀ (s)"); ax2.set_ylabel("Est. T₀ (s)")

# distance‑to‑London regression
ax3.scatter(df.loc[maskL,'dLON_true'],df.loc[maskL,'dLON_est'],s=25,alpha=.8)
xL=np.linspace(0,df.dLON_true.max(),100)
ax3.plot(xL,coefL[0]*xL+coefL[1],'r--')
ax3.set_title(f"Distance‑to‑London fit  (slope={coefL[0]:.4f}, R²={R2L:.4f})")
ax3.set_xlabel("True distance (km)"); ax3.set_ylabel("Est. distance (km)")

plt.tight_layout(); plt.show()


In [ ]:
# ============================================================
# Cell 2 – Pure‑NCD detector (global‑noise baseline, rich report)
# ============================================================

"""
Cell X – Unsupervised Stroke Detection via Normalized Compression Distance (NCD)
-------------------------------------------------------------------------------

This cell implements a fast, bit‐level, BZ2‐based anomaly detector on overlapping
windows of raw ADC data, using Normalized Compression Distance (NCD) as the
novelty score.  It then evaluates detection performance both per‐station and
at the network level against the known lightning stroke times.

Configuration Parameters
------------------------
WIN, HOP    : Window length (1024 samples ≃ 9.4 ms) and hop (512 samples = 50%
              overlap).  Balances time resolution vs. computational cost.
BASE_PCT    : Percentage of lowest‐entropy windows used to choose the “quiet”
              baseline.  Avoids hand‐picking a quiet prelude; adapts to station‐
              specific noise levels.
PCT_THR     : Percentile threshold on the NCD vector (e.g. 98.5 → top 1.5% as
              anomalies).  Captures the rare, highly‐complex windows likely
              containing lightning bursts.
Z_SIGMA     : Optional μ + Z·σ clamp on NCD to guard against long tails or
              mis‐estimated percentiles.
MIN_STN     : Minimum number of stations that must flag the same stroke to count
              it as a network‐detected event.
STN         : List of station identifiers in the array.

Core Concepts & Workflow
------------------------
1) **Sign‐Bit Representation**
   We convert each WIN‐length window of raw int16 samples into a 1‐bit‐per‐sample
   “sign‐bit delta” sequence.  This captures the waveform’s shape while throwing
   away amplitude scaling, and packs eight sign‐bits into a single byte to feed
   binary compressors.

2) **Baseline Selection via Low‐Entropy Pool**
   - Compute compressed size Cᵢ for every window’s sign‐bit bytes.
   - Select the lowest BASE_PCT % by size—i.e. the windows with most
     predictable (quiet) content.
   - From those, pick the median‐size window as the *baseline* b, with size
     C_b.  This dynamic baseline adapts to station noise and avoids assuming
     a quiet prelude.

3) **NCD Computation**
   For each window aᵢ:
       NCD(aᵢ, b) = [C(aᵢ‖b) – min(C(aᵢ), C_b)] / max(C(aᵢ), C_b)
   where C(·) is the BZ2‐compressed length.  NCD ranges [0,1+] and quantifies
   how “novel” aᵢ is compared to baseline b.

4) **Thresholding & Clamping**
   - Compute the PCT_THR‐percentile NCD value.
   - Also compute z_thr = μ + Z_SIGMA·σ for robustness against heavy tails.
   - Choose the more conservative (lower) of these two as the final threshold.
   - Mark windows with NCD > thr as “hot.”

5) **Per‐Station Reporting**
   For each station we record:
     • Baseline window index & size (bytes)
     • Min/max window compressed sizes
     • Full NCD distribution: mean, variance, median, percentiles
     • Chosen threshold values (percentile vs. z‐clamp)
     • Number & statistics (mean/σ/top‐5) of flagged windows

6) **Stroke‐By‐Stroke Evaluation**
   - For each true stroke time, compute its earliest‐arrival sample index
     across all stations (assuming c≈300 000 km/s propagation).
   - For each station s, check if any of the flagged windows within one hop
     of that sample index contains the stroke.
   - Count how many stations s report that stroke (“k‐station hits”).

7) **Performance Metrics**
   - **Per‐Station**: Confusion matrix and P/R/F1 for each station separately,
     comparing station‐level hits vs. true strokes.
   - **Network**: Treat strokes detected by ≥MIN_STN stations as true network
     detections; compute P/R/F1 accordingly.

Design Rationale
----------------
- **Unsupervised & Adaptive**: No pre‐labeled quiet segment needed; baseline
  derives from the data itself via a low‐entropy pool.
- **Bit‐Level, Compressor‐Based**: Sign‐bit deltas + BZ2 capture structural
  complexity rather than amplitude.  Compressor models long‐range patterns
  implicitly, making NCD sensitive to transient bursts.
- **Dual Thresholding**: Combines percentile and z‐score clamping to adapt
  across varying noise distributions and avoid pathological outliers.
- **Window Overlap**: 50% overlap ensures that a sharp transient straddling
  window boundaries is still fully captured in at least one window.

Strengths & Weaknesses
----------------------
+ **Strengths**
  - Works entirely unsupervised with minimal tuning per station.
  - Adapts to station‐specific noise levels and patterns.
  - Leverages general‐purpose compressors—no hand‐crafted filters or
    domain‐specific features needed.

− **Weaknesses**
  - Compressor overhead can be heavy; optimized LZMA/BZ2 settings or
    GPU‐accelerated byte‐level compressors could speed it up.
  - NCD may be insensitive to low‐amplitude strokes that mimic baseline
    entropy levels.
  - Choice of baseline percentile and compressor settings can affect
    sensitivity vs. false‐alarm rate.
  - Overlap and hop parameters trade off detection latency vs. compute load.

Potential Extensions
--------------------
- Use multi‐baseline strategy (e.g. k‐means on comp_size) to better model
  multi‐modal noise distributions.
- Explore alternate compressors (LZ4, LZMA) or learned byte‐level
  variational compressors for improved novelty sensitivity.
- Combine NCD scores with conventional features (energy, STA/LTA, spectral
  entropy) in a hybrid unsupervised anomaly detector (e.g. Isolation Forest).


The reason we convert each raw‐sample window into a “sign‐bit” sequence before feeding it to the compressor is twofold:

Focus on waveform structure, not absolute amplitude
Lightning sferics are characterized by very sharp, transient shape changes in the waveform. By taking the sign of the first‐difference (i.e. whether each sample went up or down relative to its predecessor), we strip away all the amplitude information (which compressors will otherwise focus on) and leave just the pattern of “ups” and “downs.” That makes our NCD score much more sensitive to changes in the shape of the signal—exactly the signature of a lightning burst—rather than slow variations in overall signal level or station gain drifts.

Byte‐packing for efficient off‐the‐shelf compression
Compressors like BZ2 operate on bytes. If we naively rounded or quantized the raw 16-bit samples and fed those bytes in, the compressor would spend most of its effort modeling the numeric amplitude distribution (and our windows would still be hundreds or thousands of bytes long). Instead, by converting to a 1-bit per sample sign‐delta and then using np.packbits, we collapse eight sign‐bits into a single byte. That gives us a compact, highly‐structured binary string where every bit matters for capturing the waveform’s microstructure—and is ideal for a general‐purpose compressor to exploit redundancies in “quiet” windows and blow up the size when a transient occurs.

In short:

Why “delta”? Taking the difference removes any DC offset and highlights changes rather than static levels.

Why “sign”? We want a binary feature that reflects only the direction of change.

Why pack into bytes? So we can use standard byte‐level compressors without writing a custom bit‐level compressor.

This lightweight, shape-focused representation is what makes a compressor‐based novelty score like NCD so effective at flagging the sudden, high-frequency excursions of a lightning stroke while ignoring slow, station-specific noise or baseline shifts.
"""



import numpy as np, bz2, tqdm.auto as tq
from functools import lru_cache
from collections import Counter
from scipy.stats import describe
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

# ── parameters you can tweak ─────────────────────────────────
WIN, HOP   = 1024, 512        # 9.4 ms, 50 % overlap
BASE_PCT   = 5                # use lowest‑entropy 5 % to pick baseline
PCT_THR    = 98.5             # percentile threshold on *all* windows
Z_SIGMA    = 3.5              # μ + Z σ clamp
MIN_STN    = 2                # network requirement
STN        = station_order

# ── helpers ─────────────────────────────────────────────────
@lru_cache(maxsize=None)
def c_size(b: bytes) -> int:
    return len(bz2.compress(b, 9))

def ncd(a: bytes, b: bytes, Ca: int, Cb: int) -> float:
    return (c_size(a+b) - min(Ca, Cb)) / max(Ca, Cb)

def sign_bits(arr: np.ndarray) -> bytes:
    diff = np.diff(arr.astype(np.int16), prepend=arr[0])
    return np.packbits((diff > 0).astype(np.uint8)).tobytes()

def win_view(sig: np.ndarray, W: int, H: int):
    n = (len(sig) - W) // H + 1
    return np.lib.stride_tricks.as_strided(
        sig,
        shape=(n, W),
        strides=(sig.strides[0]*H, sig.strides[0])
    )

# -------- 1) build per‑station NCD & metadata ---------------
n_win = min(((len(quantized[n]) - WIN)//HOP)+1 for n in STN)
meta   = {}             # store everything here

print(f"\nWindow = {WIN} samples  ({WIN/FS*1e3:.2f} ms)   hop = {HOP} samples")
print(f"Total windows analysed per station: {n_win:,}\n")

for nm in STN:
    sig  = quantized[nm]
    wmat = win_view(sig, WIN, HOP)

    # pass‑1: compressed size of each window
    comp_sz = np.empty(n_win, np.uint16)
    for i in tq.trange(n_win, desc=f"{nm} size pass", leave=False):
        comp_sz[i] = c_size(sign_bits(wmat[i]))

    # choose baseline = median of lowest BASE_PCT %
    k = max(1, int(BASE_PCT/100 * n_win))
    low_idx = np.argpartition(comp_sz, k)[:k]
    base_idx = low_idx[np.argsort(comp_sz[low_idx])[k//2]]
    base_b   = sign_bits(wmat[base_idx])
    Cb       = c_size(base_b)

    # pass‑2: NCD of every window vs baseline
    ncd_vec = np.empty(n_win, float)
    for i in tq.trange(n_win, desc=f"{nm} NCD pass", leave=False):
        wb = sign_bits(wmat[i])
        ncd_vec[i] = ncd(wb, base_b, comp_sz[i], Cb)

    # statistics & threshold
    stats = describe(ncd_vec)
    pct_thr = np.percentile(ncd_vec, PCT_THR)
    z_thr   = stats.mean + Z_SIGMA*stats.variance**0.5
    thr     = min(pct_thr, z_thr)
    hot     = ncd_vec > thr

    meta[nm] = dict(
        base_idx   = base_idx,
        base_size  = Cb,
        min_size   = comp_sz.min(),
        max_size   = comp_sz.max(),
        ncd        = ncd_vec,
        hot        = hot,
        thr_pct    = pct_thr,
        thr_z      = z_thr,
        thr_used   = thr,
        desc       = stats,
        hot_mu     = ncd_vec[hot].mean() if hot.any() else np.nan,
        hot_sd     = ncd_vec[hot].std(ddof=0) if hot.any() else np.nan,
        top5       = np.sort(ncd_vec)[-5:][::-1]
    )

# -------- 2) pretty report ----------------------------------
for nm in STN:
    r = meta[nm]
    print(f"\n{nm} — baseline window #{r['base_idx']}  "
          f"C={r['base_size']} B  (min={r['min_size']} B  max={r['max_size']} B)")
    print(f"     NCD: μ={r['desc'].mean:.4f}  σ={np.sqrt(r['desc'].variance):.4f}  "
          f"median={np.median(r['ncd']):.4f}  p1={np.percentile(r['ncd'],1):.4f}  "
          f"p99={np.percentile(r['ncd'],99):.4f}")
    print(f"     thr_pct={r['thr_pct']:.4f}  thr_z={r['thr_z']:.4f}  "
          f"→ thr_used={r['thr_used']:.4f}")
    print(f"     hot windows = {r['hot'].sum():,}  "
          f"μ_hot={r['hot_mu']:.4f}  σ_hot={r['hot_sd']:.4f}")
    print(f"     top‑5 NCD windows: {np.round(r['top5'],4)}")

# -------- 3) build stroke list ------------------------------
stroke_idx = [min(int((t0 + hav(ev['lat'],ev['lon'],
                                stations[n]['lat'],stations[n]['lon'])/300000)*FS)
                    for n in STN)
              for ev in events for t0 in ev['stroke_times']]
stroke_idx = np.array(stroke_idx)
truth = np.ones(len(stroke_idx), bool)

# -------- 4) per‑stroke station hits ------------------------
hits = np.zeros((len(STN), len(stroke_idx)), bool)
for s,nm in enumerate(STN):
    hot = meta[nm]['hot']
    for j,i0 in enumerate(stroke_idx):
        w = i0 // HOP
        hits[s, j] = hot[max(0, w-1):min(len(hot), w+2)].any()

cnt = hits.sum(axis=0)

print("\nStations ≥ thr per stroke:")
for k,v in sorted(Counter(cnt).items()):
    print(f"  {k} stations → {v} strokes")

# -------- 5) metrics ----------------------------------------
print("\nStation‑level stroke detection:")
print("stn  TP  FP  FN   P      R      F1")
for s,nm in enumerate(STN):
    pred = hits[s]
    tn,fp,fn,tp = confusion_matrix(truth, pred, labels=[False,True]).ravel()
    P = precision_score(truth, pred, zero_division=0)
    R = recall_score   (truth, pred, zero_division=0)
    F = f1_score       (truth, pred, zero_division=0)
    print(f"{nm:>3} {tp:3d} {fp:3d} {fn:3d}  {P:6.3f} {R:6.3f} {F:6.3f}")

net_pred = cnt >= MIN_STN
tn,fp,fn,tp = confusion_matrix(truth, net_pred, labels=[False,True]).ravel()
P_net = precision_score(truth, net_pred, zero_division=0)
R_net = recall_score   (truth, net_pred, zero_division=0)
F_net = f1_score       (truth, net_pred, zero_division=0)
print(f"\nNetwork (≥{MIN_STN} stn)  TP={tp}  FP={fp}  FN={fn}  TN={tn}")
print(f"P={P_net:.3f}  R={R_net:.3f}  F1={F_net:.3f}")


In [ ]:
# ============================================================
# Cell 3 – Isolation‑Forest detector with rich signal features
#           (STA/LTA • Hilbert • spectral • wavelet • compress)
#           50 %‑overlap windows, per‑station & network scores
# ============================================================
"""
Unsupervised Lightning Stroke Detection via Handcrafted Time‐Frequency‐Complexity Features
and Per-Station Isolation Forests
-----------------------------------------------------------------------------------------

This module implements a fully unsupervised pipeline to detect individual lightning
strokes in multi-station ADC waveforms by combining a diverse set of handcrafted
features with robust anomaly detection.  It operates station-by-station, then
coincides detections across the network to identify true strokes.

Key Steps
=========
1. Parameters
   - WIN, HOP: 50% overlapping window parameters (1024 samples ≃ 9.4 ms windows).
   - CONTAM: Target contamination rate per station (~0.3% of windows flagged).
   - MIN_STN, TOL_WIN: Network voting threshold and tolerance for stroke coincidence.

2. Helper Functions
   - sta_lta: Short-term average / long-term average ratio, emphasizing sudden
     amplitude rises.
   - crest_factor: Peak/RMS ratio over a short central sub-segment, capturing
     waveform “spikiness.”
   - comp_ratio: Byte-level compression ratio (zlib) of the raw int16 samples,
     measuring structural novelty.
   - Hilbert envelope & FFT spectral analysis used inline below.

3. Feature Extraction (13 per window)
   - **Hilbert Envelope**:
       1) peak_env      : max(|analytic signal|)
       2) med_env       : median(|analytic signal|)
       3) ratio_env     : peak_env/med_env
   - **Time-Domain Energy**:
       4) energy        : ∑ x² over window
       5) sta_lta       : ratio of mean(|env|) over STA vs. LTA
   - **Crest Factors**:
       6) cf_short      : peak/RMS in a WIN/8 centered segment
       7) cf_global     : overall peak / global RMS
   - **FFT Band-Power Fractions**:
       8–11) frac1…4    : normalized power in four equally-spaced sub-bands of the FFT
   - **Wavelet & Compression**:
       12) wave_hi     : energy fraction in first detail level of a 3-level 'db4' wavelet
       13) comp_r      : zlib compressed size ÷ raw byte length

4. Per-Station Modeling
   - StandardScaler: zero-mean, unit-variance across windows.
   - IsolationForest: C++-backed, n_estimators=150, contamination=CONTAM.
   - Windows with anomaly label (−1) are flagged as potential stroke windows.

5. Stroke Enumeration & Evaluation
   - `stroke_samples` provide the true sample indices (earliest arrival across stations).
   - A station “votes” if it flagged any window within ±TOL_WIN of the stroke index.
   - We report station-level TP/FP/FN/P/R/F1 and network-level metrics (requiring ≥MIN_STN).

Strengths
=========
- **Feature Diversity**: captures transient, spectral, multi-scale, and complexity cues.
- **Unsupervised**: no stroke labels needed for training; contamination controls sensitivity.
- **Per-Station Calibration**: each station adapts to its own noise floor and idiosyncrasies.

Weaknesses & Extensions
=======================
- **Handcrafted Limitation**: fixed windows may misalign partial events; feature set may
  miss subtle correlated patterns across stations.
- **IsolationForest Assumptions**: assumes windows are i.i.d outliers in feature space.
- **Scaling Sensitivity**: global StandardScaler may be influenced by extreme outliers.
- **Fixed Contamination**: more robust would be adaptive contamination (e.g. via Q-statistics).
- **Window Boundaries**: consider sliding windows with soft voting or event merging.

Potential Improvements
======================
1. **Dynamic Noise Estimation**: adopt per-station noise models to adapt thresholds.
2. **Graph-based Models**: learn joint spatial–temporal patterns across station network
   using graph neural networks (GNNs) or spectral graph filters.
3. **Deep Representations**: replace handcrafted features + IsolationForest with
   convolutional variational autoencoders or Deep SVDD for non-linear anomaly scoring.
4. **Semi-Supervised Tuning**: if a small set of labelled strokes exists, calibrate
   detector via one-class SVM or fine-tune deep models for higher fidelity.
5. **Post-Processing**: merge adjacent flagged windows and refine based on
   continuation across multiple stations.

Usage
=====
Simply run this cell after computing `quantized`, `stroke_samples`, and `stroke_truth`
from the earlier data-generation and Hilbert-envelope detection steps.  Results will
print concise station- and network-level detection metrics.
"""



import numpy as np, zlib, pywt, math, sys
from scipy.signal import hilbert, lfilter
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    precision_score, recall_score, f1_score, confusion_matrix
)
from collections import Counter
from tqdm.auto import tqdm

# ── parameters (reuse values from previous cells) ────────────
WIN       = 1024
HOP       = WIN // 2
STATIONS  = station_order
FS        = float(FS)
TOL_WIN   = 1
MIN_STN   = 2
CONTAM    = 0.003          # ≈0.3 % windows flagged per station

# ------------------------------------------------------------
def sta_lta(x: np.ndarray, sta=128, lta=1024):
    """Return STA/LTA ratio for centre of x (padding with edge value)."""
    if len(x) < lta:          # shouldn’t happen – WIN >= LTA
        return 1.0
    c   = len(x) // 2
    sta_mean = x[c-sta//2:c+sta//2].astype(float).mean()
    lta_mean = x[c-lta//2:c+lta//2].astype(float).mean()
    return sta_mean / (lta_mean + 1e-9)

def crest_factor(seg: np.ndarray):
    """peak / RMS over short (WIN/8) segment centred in window."""
    n = len(seg) // 8
    c = len(seg) // 2
    part = seg[c-n//2:c+n//2].astype(float)
    rms  = math.sqrt((part**2).mean()) + 1e-9
    return np.abs(part).max() / rms

def comp_ratio(seg: np.ndarray) -> float:
    raw  = seg.tobytes()
    comp = zlib.compress(raw, 6)
    return len(comp) / (len(raw) if len(raw) else 1)

# ------------------------------------------------------------
print(f"Building feature matrices...  WIN={WIN}, HOP={HOP}, overlap=50 %")
n_win = min(((len(quantized[n]) - WIN)//HOP)+1 for n in STATIONS)
feat_dim = 13         # we will collect 13 features / window
features = {nm: np.empty((n_win, feat_dim), dtype=float) for nm in STATIONS}

for nm in STATIONS:
    sig = quantized[nm]
    env = np.abs(hilbert(sig.astype(float)))
    Nfft = WIN // 2 + 1
    b25 = int(Nfft*0.25); b50=int(Nfft*0.50); b75=int(Nfft*0.75)
    for w in tqdm(range(n_win), desc=f"{nm} windows", leave=False):
        s = w * HOP
        seg_i16 = sig[s:s+WIN]
        seg_f   = seg_i16.astype(float)
        env_seg = env[s:s+WIN]

        # 1‑3  : Hilbert envelope (peak, median, peak/median)
        peak_env = env_seg.max()
        med_env  = np.median(env_seg)
        ratio_env= peak_env/(med_env+1e-9)

        # 4‑5  : Energy + STA/LTA
        energy = float((seg_f**2).sum())
        stalta = sta_lta(env_seg)

        # 6‑7  : Crest factor (short segment) + global crest factor
        cf_short = crest_factor(seg_i16)
        cf_global= peak_env / (math.sqrt((seg_f**2).mean())+1e-9)

        # 8‑11 : FFT band‑power fractions
        P = np.abs(np.fft.rfft(seg_f))**2
        totP = P.sum() + 1e-9
        frac1 = P[:b25].sum()/totP
        frac2 = P[b25:b50].sum()/totP
        frac3 = P[b50:b75].sum()/totP
        frac4 = P[b75:].sum()/totP

        # 12‑13: Wavelet high/low frac + compression ratio
        coeffs = pywt.wavedec(seg_f, 'db4', level=3)
        details = coeffs[1:]     # D1‑D3
        highE = (details[0]**2).sum()
        lowE  = (details[-1]**2).sum()
        totE  = highE + lowE + 1e-9
        wave_hi = highE / totE
        comp_r  = comp_ratio(seg_i16)

        features[nm][w] = [
            peak_env, med_env, ratio_env,
            energy, stalta,
            cf_short, cf_global,
            frac1, frac2, frac3, frac4,
            wave_hi, comp_r
        ]

# ------------------------------------------------------------
print("\nFitting Isolation Forest per station...")
hot = {}
for nm in STATIONS:
    X   = features[nm]
    # z‑score scaling to unit variance
    scaler = StandardScaler().fit(X)
    Xs = scaler.transform(X)
    iso = IsolationForest(
        n_estimators=150,
        max_samples='auto',
        contamination=CONTAM,
        bootstrap=False,
        random_state=42
    ).fit(Xs)
    yhat = iso.predict(Xs)   # -1 = anomaly
    hot[nm] = (yhat == -1)
    print(f" {nm}: windows flagged = {hot[nm].sum():5d} / {n_win} "
          f"(contam={CONTAM:.3%})")

# ------------------------------------------------------------
# stroke_samples, stroke_truth come from Cell 2
counts = np.zeros(len(stroke_samples), int)
for nm in STATIONS:
    m = hot[nm]
    for j,i0 in enumerate(stroke_samples):
        w = i0 // HOP
        if m[max(0,w-TOL_WIN):min(n_win,w+TOL_WIN+1)].any():
            counts[j] += 1

print("\nStations ≥thr per stroke:")
for k,v in sorted(Counter(counts).items()):
    print(f"  {k:2d} stations → {v} strokes")

print("\nStation‑level stroke detection:")
print("stn TP FP FN     P      R     F1")
for nm in STATIONS:
    pred = np.array([hot[nm][max(0,min(n_win-1,i0//HOP))] for i0 in stroke_samples])
    tn,fp,fn,tp = confusion_matrix(stroke_truth,pred,labels=[False,True]).ravel()
    P = precision_score(stroke_truth,pred,zero_division=0)
    R = recall_score   (stroke_truth,pred,zero_division=0)
    F = f1_score       (stroke_truth,pred,zero_division=0)
    print(f"{nm:>3} {tp:2d} {fp:2d} {fn:2d}  {P:6.3f} {R:6.3f} {F:6.3f}")

net_pred = counts >= MIN_STN
tn,fp,fn,tp = confusion_matrix(stroke_truth,net_pred,labels=[False,True]).ravel()
P_net = precision_score(stroke_truth,net_pred,zero_division=0)
R_net = recall_score   (stroke_truth,net_pred,zero_division=0)
F_net = f1_score       (stroke_truth,net_pred,zero_division=0)
print(f"\nNetwork (≥{MIN_STN} stn) stroke‑wise:")
print(f" TP={tp} FP={fp} FN={fn} TN={tn}   P={P_net:.3f} R={R_net:.3f} F1={F_net:.3f}")


In [ ]:
# ============================================================
# Cell 3 – Isolation‑Forest with Robust scaling + extra spectra
# ============================================================
"""
Unsupervised Lightning Stroke Detection with Expanded Feature Set and Robust Scaling
------------------------------------------------------------------------------------

This cell implements a station‐by‐station anomaly detection pipeline using an
Isolation Forest on a rich set of 16 handcrafted features, scaled via a robust
scaler to mitigate the influence of outliers.  We allow per-station contamination
tuning to account for differing noise levels, and then combine station flags
to identify true strokes across the network.

Configuration
-------------
WIN, HOP      : 1024 sample windows with 50% overlap (≈9.4 ms window / 512 sample hop)
STATIONS      : List of station codes (e.g. ['LON','LER','DUB','BER'])
FS            : Sampling rate carried forward from Cell 1
TOL_WIN       : ±1 window tolerance for stroke‐flag coincidence
BASE_CONT     : Base contamination rate per station (0.3 %)
MIN_STN       : Minimum number of stations required to declare a stroke
N_EST         : Number of trees in each Isolation Forest (150)

Helper Functions
----------------
sta_lta(env, sta=128, lta=1024)
    Computes the short‐term average / long‐term average ratio of the Hilbert
    envelope, highlighting sudden transients.

crest(seg)
    Computes peak/RMS ratio (“crest factor”) over the full window.

comp_ratio(seg)
    Byte‐level compression ratio (zlib) of the raw int16 samples, a measure
    of local structural complexity.

spec_stats(seg_f)
    Computes spectral centroid, bandwidth, and Shannon entropy of the power
    spectrum |FFT|² to capture frequency‐domain signatures.

Feature Extraction (16 per window)
----------------------------------
 1) peak_env      – maximum of |analytic signal|
 2) med_env       – median of |analytic signal|
 3) ratio_env     – peak_env / med_env
 4) energy        – total energy ∑ x²
 5) sta_lta       – envelope STA/LTA ratio
 6) crest_short   – crest factor over central WIN/8 segment
 7) crest_glob    – crest factor over full window
 8–11) b1…b4      – normalized FFT power fractions in four equal sub-bands
 12) wave_hi      – high‐frequency energy fraction from level-3 'db4' wavelet
 13) comp_r       – compression size / raw size
 14) centroid     – spectral centroid (Hz)
 15) bw           – spectral bandwidth
 16) ent          – spectral entropy

Per‐Station Modeling
--------------------
1. Extract the 16‐D feature matrix X  (n_win × 16) for each station.
2. Apply `RobustScaler` (center-median, scale-IQR) to reduce sensitivity to extreme outliers.
3. Choose contamination:
     - LON, LER: use BASE_CONT (0.3 %)
     - Others: allow a higher contamination (1.5×) to handle weaker signals.
4. Fit `IsolationForest(n_estimators=N_EST, contamination=contam, random_state=42)`.
5. Flag windows where `predict(Xs) == -1`.

Stroke Coincidence Logic
------------------------
- For each true stroke sample index `i0`, compute its window index `w = i0 // HOP`.
- A station “votes” if it flagged any window in `[w−TOL_WIN, w+TOL_WIN]`.
- Tally station votes per stroke; network declares a stroke if ≥ MIN_STN stations voted.

Reporting
---------
- Prints per‐station counts of flagged windows vs window total.
- Prints station‐level TP, FP, FN, and P/R/F1 against ground‐truth stroke times.
- Prints network‐level TP/FP/FN/TN and overall P/R/F1 for ≥MIN_STN coincidence.

Strengths
---------
- **Robust Scaling** mitigates influence of extreme noise bursts.
- **Feature Diversity** spans time‐domain, frequency‐domain, wavelet, and compression cues.
- **Per‐Station Contamination Tuning** adapts to station‐specific noise floors.

Weaknesses & Possible Extensions
--------------------------------
- Manual contamination rates may be suboptimal; could be chosen via cross‐validation
  on a small labeled set or using statistical thresholds.
- IsolationForest assumes window independence; structured deep models (e.g. autoencoders
  or graph neural nets) may exploit temporal or inter‐station correlations.
- Fixed window size may miss very short or long transients—consider multi‐scale windows.

Usage
-----
Simply run this cell after loading `quantized`, `stroke_samples`, and `stroke_truth`
from earlier cells.  The results will print station‐ and network‐level detection metrics.
"""


import numpy as np, zlib, pywt, math, sys
from scipy.signal import hilbert
from sklearn.preprocessing import RobustScaler          # ► changed
from sklearn.ensemble import IsolationForest
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from tqdm.auto import tqdm
from collections import Counter

# ---------- configuration (same as before) ------------------
WIN, HOP   = 1024, 512
STATIONS   = station_order
FS         = float(FS)
TOL_WIN    = 1
BASE_CONT  = 0.003          # default contamination
MIN_STN    = 2
N_EST      = 150

# ---------- helper functions --------------------------------
def sta_lta(env, sta=128, lta=1024):
    c = len(env)//2
    return env[c-sta//2:c+sta//2].mean() / (env[c-lta//2:c+lta//2].mean()+1e-9)

def crest(seg):
    rms = math.sqrt((seg.astype(float)**2).mean()) + 1e-9
    return np.abs(seg).max()/rms

def comp_ratio(seg):
    raw = seg.tobytes()
    return len(zlib.compress(raw,6))/len(raw)

def spec_stats(seg_f):
    """return centroid, bandwidth and entropy over |FFT|²"""
    P = np.abs(np.fft.rfft(seg_f))**2
    P /= P.sum()+1e-12
    freqs = np.fft.rfftfreq(len(seg_f), d=1/FS)
    centroid = (freqs*P).sum()
    bandwidth= math.sqrt(((freqs-centroid)**2*P).sum())
    entropy  = -(P*np.log2(P+1e-12)).sum()
    return centroid, bandwidth, entropy

# ---------- feature extraction -------------------------------
print(f"Feature list:")
print("  peak_env, med_env, ratio_env, energy, sta/lta, "
      "crest_short, crest_glob, band1..4, wave_hi, comp_r, "
      "centroid, bw, ent")

n_win = min(((len(quantized[n]) - WIN)//HOP)+1 for n in STATIONS)
feat_dim = 16   # 13 old + 3 spectral stats
X_station = {nm: np.empty((n_win, feat_dim), float) for nm in STATIONS}

for nm in STATIONS:
    sig = quantized[nm]
    env = np.abs(hilbert(sig.astype(float)))
    Nfft = WIN//2+1
    borders = [int(Nfft*r) for r in (0.25,0.50,0.75)]
    for w in tqdm(range(n_win), desc=f"{nm} windows", leave=False):
        s = w*HOP
        seg_i16 = sig[s:s+WIN]
        seg_f   = seg_i16.astype(float)
        env_seg = env[s:s+WIN]

        # Envelope features
        peak_env = env_seg.max(); med_env = np.median(env_seg)
        ratio_env= peak_env/(med_env+1e-9)

        # Energy & STA/LTA
        energy = (seg_f**2).sum(); stalta = sta_lta(env_seg)

        # Crest factors
        crest_s = crest(seg_i16[len(seg_i16)//2-WIN//16:len(seg_i16)//2+WIN//16])
        crest_g = crest(seg_i16)

        # FFT band‑power fractions
        P = np.abs(np.fft.rfft(seg_f))**2
        totP = P.sum()+1e-9
        f1,f2,f3 = borders
        b1 = P[:f1].sum()/totP; b2=P[f1:f2].sum()/totP
        b3 = P[f2:f3].sum()/totP; b4=P[f3:].sum()/totP

        # Wavelet & compression
        hi = pywt.wavedec(seg_f,'db4',level=3)[1]; lo = pywt.wavedec(seg_f,'db4',level=3)[-1]
        wave_hi = (hi**2).sum()/((hi**2).sum()+(lo**2).sum()+1e-9)
        comp_r = comp_ratio(seg_i16)

        # Spectral centroid / bandwidth / entropy
        cent, bw, ent = spec_stats(seg_f)

        X_station[nm][w] = [
            peak_env, med_env, ratio_env,
            energy, stalta,
            crest_s, crest_g,
            b1,b2,b3,b4,
            wave_hi, comp_r,
            cent, bw, ent
        ]

# ---------- fit Isolation Forest per station ----------------
hot = {}
for nm in STATIONS:
    X = X_station[nm]
    rob = RobustScaler().fit(X)        # ► changed scaler
    Xs  = rob.transform(X)

    # OPTIONAL: allow weak stations a bit more contamination
    contam = BASE_CONT if nm in ('LON','LER') else BASE_CONT*1.5

    iso  = IsolationForest(
              n_estimators=N_EST,
              contamination=contam,
              random_state=42
           ).fit(Xs)
    hot[nm] = (iso.predict(Xs) == -1)
    print(f"{nm}: windows flagged = {hot[nm].sum():4d} / {n_win} (contam {contam:.3%})")

# ---------- per‑stroke scoring ------------------------------
counts = np.zeros(len(stroke_samples), int)
for nm in STATIONS:
    m = hot[nm]
    for j,i0 in enumerate(stroke_samples):
        w = i0//HOP
        if m[max(0,w-TOL_WIN):min(n_win,w+TOL_WIN+1)].any():
            counts[j]+=1

print("\nStations ≥thr per stroke:")
for k,v in sorted(Counter(counts).items()):
    print(f"  {k:2d} stations → {v} strokes")

print("\nStation‑level stroke detection:")
print("stn TP FP FN     P      R     F1")
for nm in STATIONS:
    pred = np.array([hot[nm][max(0,min(n_win-1,i0//HOP))] for i0 in stroke_samples])
    tn,fp,fn,tp = confusion_matrix(stroke_truth,pred,labels=[False,True]).ravel()
    P = precision_score(stroke_truth,pred,zero_division=0)
    R = recall_score   (stroke_truth,pred,zero_division=0)
    F = f1_score       (stroke_truth,pred,zero_division=0)
    print(f"{nm:>3} {tp:2d} {fp:2d} {fn:2d}  {P:6.3f} {R:6.3f} {F:6.3f}")

net_pred = counts >= MIN_STN
tn,fp,fn,tp = confusion_matrix(stroke_truth,net_pred,labels=[False,True]).ravel()
P_net=precision_score(stroke_truth,net_pred,zero_division=0)
R_net=recall_score   (stroke_truth,net_pred,zero_division=0)
F_net=f1_score       (stroke_truth,net_pred,zero_division=0)
print(f"\nNetwork (≥{MIN_STN} stn) stroke‑wise:")
print(f" TP={tp} FP={fp} FN={fn} TN={tn}   P={P_net:.3f} R={R_net:.3f} F1={F_net:.3f}")


In [ ]:
# ============================================================
# Cell 3‑EIF‑isotree  (API‑robust)
# ============================================================

"""
Extended Isolation Forest (EIF) Lightning Stroke Detector
--------------------------------------------------------

This cell upgrades the previous station‐wise Isolation Forest approach by
leveraging the fast, scalable Extended Isolation Forest (EIF) implementation
(`isotree.IsolationForest`), along with adaptive contamination selection and
an extreme‐value fallback step.  These enhancements yield more robust,
fine‐grained anomaly scores and better handle stations with varying noise
characteristics.

Key Differences & Improvements
------------------------------
1. **Extended Isolation Forest (EIF)**
   - Uses the C++/OpenMP‐based `isotree` package for much faster tree building
     and scoring on large feature sets.
   - Supports direct “average depth” extraction (`avg_depth`), giving a
     continuous anomaly measure where _lower_ = more anomalous.

2. **Adaptive Contamination Selection (`GRID_CONT`)**
   - Instead of a fixed contamination rate, we sweep a small grid of candidate
     rates (0.3%–0.7%) per station.
   - Choose the _smallest_ contamination that still flags ≥0.1% of windows,
     ensuring we don’t over‐flag stations with very low anomaly rates.

3. **Extreme‐Value Fallback (`EXTREME_Q`)**
   - For strokes missed by all stations at the normal threshold, apply a
     second, more extreme quantile (99.95th percentile) per station.
   - Recovers rare true positives that might lie below the standard contamination
     cutoff.

4. **Full Threading & Tunable Dimensions**
   - `ntrees=200` for deeper ensemble coverage.
   - `ndim = feature_dim − 1` to enable multi‐feature splits without bias.
   - Automatic parallelism (`nthreads = cpu_count−1`) maximises throughput.

Pipeline Overview
-----------------
1. **Feature Extraction**
   - Same rich 16‐dimensional feature vector per window as prior (envelope,
     energy, STA/LTA, crest factors, FFT bands, wavelet energy, compression,
     spectral stats).

2. **EIF Fitting & Scoring**
   - Scale features with `RobustScaler` to reduce extreme‐value influence.
   - Fit one EIF per station, extract per‐window depth scores (`score`).

3. **Contamination Grid Search**
   - For each candidate `c` in `GRID_CONT`, compute threshold = quantile(score, c).
   - Select first `c` where flagged windows ≥ 0.1% of total, store mask.

4. **Per‐Stroke Voting**
   - As before, convert stroke sample indices to window indices ±1 (`TOL_WIN`).
   - Count station votes; require ≥ `MIN_STN` stations to declare stroke.
   - Apply extreme‐value fallback for any unflagged strokes using `EXTREME_Q`.

5. **Metrics Reporting**
   - Print per‐station chosen contamination and flagged‐window counts.
   - Print station‐level TP/FN and recall, plus network‐level P/R/F1.

Why It May Be Better
---------------------
- **Speed & Scalability**: EIF’s C++ core and multi‐threading sharply reduces
  runtime on large datasets.
- **Adaptive Sensitivity**: Grid‐search contamination tailors anomaly rates per
  station, avoiding under/over‐flagging.
- **Robust Recovery**: Extreme‐quantile fallback step catches rare true strokes
  otherwise missed.
- **Flexible Depth Scoring**: Continuous “average depth” provides finer confidence
  measures than binary anomaly classes.

Usage
-----
Run this cell after feature extraction, then proceed to the downstream
per‐stroke evaluation.  The printed metrics will reflect the improved EIF
pipeline’s performance and network‐level detection accuracy.
"""


import os, warnings, zlib, math, numpy as np, pywt
from math import sqrt
from scipy.signal import hilbert
from tqdm.auto import tqdm
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support
from isotree import IsolationForest
warnings.filterwarnings("ignore")

# ------------ parameters ------------------------------------
WIN, HOP  = 1024, 512
STN       = station_order
FS        = float(FS)
TOL_WIN   = 1
MIN_STN   = 2
GRID_CONT = np.linspace(0.003, 0.007, 5)
EXTREME_Q = 99.95
NTREES    = 200

# ------------ helpers ---------------------------------------
def sta_lta(env, sta=128, lta=1024):
    c=len(env)//2
    return env[c-sta//2:c+sta//2].mean()/(env[c-lta//2:c+lta//2].mean()+1e-9)
def crest(seg): rms=sqrt((seg.astype(float)**2).mean())+1e-9; return np.abs(seg).max()/rms
comp = lambda seg: len(zlib.compress(seg.tobytes(),6))/len(seg.tobytes())
def spec_stats(seg):
    P=np.abs(np.fft.rfft(seg))**2; P/=P.sum()+1e-12
    f=np.fft.rfftfreq(len(seg),1/FS)
    cent=(f*P).sum(); bw=sqrt(((f-cent)**2*P).sum()); ent=-(P*np.log2(P+1e-12)).sum()
    return cent,bw,ent

def get_depth_or_score(iso, X):
    """
    Return “depth” so that *lower* means “more anomalous”,
    regardless of isotree version.
    """
    try:
        return iso.predict(X, type="avg_depth"), True     # new API
    except TypeError:
        try:
            return iso.predict(X, output_type="avg_depth"), True
        except TypeError:
            # fall back to anomaly score (higher = more anomalous)
            return iso.predict(X), False

# ------------ feature extraction ----------------------------
n_win=min(((len(quantized[n])-WIN)//HOP)+1 for n in STN)
feat_dim=16
Xst={nm: np.empty((n_win, feat_dim), float) for nm in STN}

print("▶ extracting features …")
for nm in STN:
    sig=quantized[nm]; env=np.abs(hilbert(sig.astype(float)))
    Nf=WIN//2+1; b25,b50,b75=[int(Nf*r) for r in (0.25,0.50,0.75)]
    for w in tqdm(range(n_win), desc=f"{nm}", leave=False):
        s=w*HOP; seg_i16=sig[s:s+WIN]; seg_f=seg_i16.astype(float); env_seg=env[s:s+WIN]
        pk,md=env_seg.max(),np.median(env_seg); ratio=pk/(md+1e-9)
        energy=(seg_f**2).sum(); stl=sta_lta(env_seg)
        cf_s=crest(seg_i16[len(seg_i16)//2-WIN//16:len(seg_i16)//2+WIN//16]); cf_g=crest(seg_i16)
        P=np.abs(np.fft.rfft(seg_f))**2; tot=P.sum()+1e-9
        frac=(P[:b25].sum()/tot,P[b25:b50].sum()/tot,P[b50:b75].sum()/tot,P[b75:].sum()/tot)
        hi=pywt.wavedec(seg_f,'db4',level=3)[1]; lo=pywt.wavedec(seg_f,'db4',level=3)[-1]
        wave_hi=(hi**2).sum()/((hi**2).sum()+(lo**2).sum()+1e-9)
        comp_r=comp(seg_i16); cent,bw,ent=spec_stats(seg_f)
        Xst[nm][w]=[pk,md,ratio,energy,stl,cf_s,cf_g,*frac,wave_hi,comp_r,cent,bw,ent]

# ------------ EIF per station -------------------------------
eif_score, hot, best_cont = {}, {}, {}
for nm in STN:
    X=RobustScaler().fit_transform(Xst[nm])
    iso=IsolationForest(
            ntrees      = NTREES,
            sample_size = 'auto',
            ndim        = X.shape[1]-1,
            prob_pick_avg_gain=0, prob_pick_pooled_gain=0,
            nthreads    = max(os.cpu_count()-1,1),
            random_seed = 42
        ).fit(X)

    score, is_depth = get_depth_or_score(iso, X)
    # flip if we got anomaly score (higher=bad)
    if not is_depth:
        score = -score          # so more negative = more anomalous

    eif_score[nm]=score
    for c in GRID_CONT:
        thr=np.quantile(score, c)
        mask=score<thr
        if mask.sum()>=0.001*n_win:
            best_cont[nm]=c; hot[nm]=mask; break
    else:
        best_cont[nm]=GRID_CONT[-1]; hot[nm]=score<np.quantile(score,GRID_CONT[-1])

    print(f"{nm}: contamination={best_cont[nm]:.3%}, flagged={hot[nm].sum()} windows")

# ------------ per‑stroke evaluation -------------------------
counts=np.zeros(len(stroke_samples),int)
for nm in STN:
    m=hot[nm]
    for j,i0 in enumerate(stroke_samples):
        w=i0//HOP
        if m[max(0,w-TOL_WIN):min(n_win,w+TOL_WIN+1)].any():
            counts[j]+=1

ext_thr={nm: np.percentile(eif_score[nm], 100-EXTREME_Q) for nm in STN}
for j,i0 in enumerate(stroke_samples):
    if counts[j]==0:
        w=i0//HOP
        for nm in STN:
            if eif_score[nm][w] < ext_thr[nm]:
                counts[j]=1; break

print("\nStations flagged per stroke:")
for k,v in sorted(dict(zip(*np.unique(counts,return_counts=True))).items()):
    print(f" {k:2d} stations → {v} strokes")

truth=np.ones(len(stroke_samples),bool)
print("\nStation‑level stroke detection:")
print("stn TP FN  Recall")
for nm in STN:
    pred=np.array([hot[nm][max(0,min(n_win-1,i0//HOP))] for i0 in stroke_samples])
    tn,fp,fn,tp=confusion_matrix(truth,pred,labels=[False,True]).ravel()
    R=tp/(tp+fn) if tp+fn else 0
    print(f"{nm:>3} {tp:2d} {fn:2d}  {R:6.3f}")

net_pred=counts>=MIN_STN
tn,fp,fn,tp=confusion_matrix(truth,net_pred,labels=[False,True]).ravel()
P,R,F=precision_recall_fscore_support(truth,net_pred,average='binary',zero_division=0)[:3]
print(f"\nNetwork (≥{MIN_STN} stn): TP={tp}  FN={fn}   P={P:.3f}  R={R:.3f}  F1={F:.3f}")


In [ ]:
# ============ EXTENDED REPORT (append at end of cell) ==================

# -----------------------------------------------------------------------
# 1) Full confusion‑matrix metrics
# -----------------------------------------------------------------------
print("\n── Station‑level confusion matrix ─────────────────────────")
hdr = "stn  TP  FP  FN  TN    P      R     F1"
print(hdr)
fmt = "{:>3} {:3d} {:3d} {:3d} {:5d}  {:6.3f} {:6.3f} {:6.3f}"
for nm in STN:
    pred = np.array([hot[nm][max(0, min(n_win-1, i0//HOP))] for i0 in stroke_samples])
    tn, fp, fn, tp = confusion_matrix(truth, pred, labels=[False, True]).ravel()
    P, R, F = precision_recall_fscore_support(
        truth, pred, average='binary', zero_division=0
    )[:3]
    print(fmt.format(nm, tp, fp, fn, tn, P, R, F))

net_pred = counts >= MIN_STN
tn, fp, fn, tp = confusion_matrix(truth, net_pred, labels=[False, True]).ravel()
Pnet, Rnet, Fnet = precision_recall_fscore_support(
    truth, net_pred, average='binary', zero_division=0
)[:3]
print("\nNetwork (≥{} stn):".format(MIN_STN))
print(fmt.format("NET", tp, fp, fn, tn, Pnet, Rnet, Fnet))

# -----------------------------------------------------------------------
# 2) Depth‑score diagnostics
# -----------------------------------------------------------------------
print("\n── Detailed station diagnostics ──────────────────────────")
for nm in STN:
    sc   = eif_score[nm]
    flag = hot[nm].sum()
    pct  = flag / n_win * 100
    print(f"\n[{nm}]")
    print(f"  chosen contamination       : {best_cont[nm]:.3%}")
    print(f"  windows flagged            : {flag} / {n_win}  ({pct:.2f} %)")
    print(f"  depth‑score range (all)    : {sc.min():.3f}  … {sc.max():.3f}")
    print(f"  depth‑score mean / σ       : {sc.mean():.3f}  /  {sc.std(ddof=0):.3f}")
    top5 = np.round(np.sort(sc)[:5], 5)
    print(f"  depth‑score top‑5 (most anomalous windows):\n     {top5}")

# -----------------------------------------------------------------------
# 3) Feature set recap
# -----------------------------------------------------------------------
feat_names = [
    "peak_env", "med_env", "ratio_env", "energy", "STA/LTA",
    "crest_short", "crest_global",
    "band_0‑0.25", "band_0.25‑0.5", "band_0.5‑0.75", "band_0.75‑1",
    "wave_hi", "comp_ratio",
    "spectrum_centroid", "spec_bw", "spec_entropy"
]
print("\n── Feature set recap ─────────────────────────────────────")
print("Features (16):", ", ".join(feat_names))

# -----------------------------------------------------------------------
# 4) Runtime / resource summary
# -----------------------------------------------------------------------
audio_ms = n_win * WIN / FS * 1e3
print("\n── Runtime summary ───────────────────────────────────────")
print(f" Feature extraction time (audio processed) : ≈ {audio_ms:.1f} ms")
print(f" Trees per station                         : {NTREES}")
print(f" Threads used (OpenMP)                     : {max(os.cpu_count()-1,1)}")


In [ ]:
# ============================================================
# Cell 3 — C‑DAE unsupervised detector (PyTorch)
#           1024‑samp windows, 50 % overlap
#           full station & network report
# ============================================================
#  Requirements:  pip install torch tqdm pywt
# ============================================================

"""
Convolutional Denoising Autoencoder (C-DAE) Lightning Stroke Detector
---------------------------------------------------------------------

This cell implements a station-wise unsupervised detector using a shallow
convolutional denoising autoencoder (C-DAE) in PyTorch.  Each station’s
continuous 1 D waveform is split into overlapping 1,024-sample windows,
which are auto-encoded and flagged as anomalous if their reconstruction
error exceeds the 99.7th percentile.  Coincident flags across ≥2 stations
are then aggregated to detect lightning strokes.

Model Architecture & Rationale
------------------------------
- **Encoder**
  Three 1 D convolutional layers (filters: 1→8→16→32, kernel size=7,
  stride=2, padding=3) progressively downsample the 1,024-sample input
  to 32 channels × 128 time steps.  A final linear layer maps
  32×128→LATENT (32 D) latent vector.
- **Decoder**
  A linear “bottleneck” expansion (32→32×128) followed by three
  transposed‐conv layers mirroring the encoder reconstructs the
  original waveform shape.

Key Design Choices
------------------
1. **Denoising Objective**
   - Inputs are corrupted with Gaussian noise (σ≈0.02) to force the
     latent code to capture robust waveform structure (sferic bursts)
     rather than overfit stationary noise.
2. **Shallow Depth & Small Latent**
   - A compact LATENT=32 forces extreme compression, making true
     lightning‐burst windows (high‐frequency, high‐amplitude)
     stand out as high‐MSE anomalies.
3. **Overlap Windows (50%)**
   - Improves temporal resolution and reduces boundary artifacts.
4. **Percentile Thresholding (99.7%)**
   - Unsupervised: flags the smallest ∼0.3 % of windows per station,
     assumed to contain bursts.

Training & Scoring
-----------------
- **Training**
  - For each station, randomly sample up to 20 000 windows.
  - Train for 4 epochs, batch size = 256, Adam lr=1e-3.
- **Inference**
  - Score every window by its mean squared reconstruction error (MSE).
  - Flag windows with MSE > PCT_THR percentile as anomalies.

Aggregation & Evaluation
------------------------
- **Stroke Alignment**
  Known true stroke times (from simulator) are converted to window
  indices (±1 tolerance).  For each stroke, count how many stations
  flagged the corresponding window.  Require ≥MIN_STN (2) stations
  to declare a detected stroke.
- **Metrics**
  - Station‐level: confusion matrix (TP/FP/FN/TN), precision, recall, F1.
  - Network‐level: aggregated P/R/F1 across strokes.

What the C-DAE Learns & Captures
--------------------------------
- **Signal Morphology**
  Learns typical noise‐floor waveform patterns; high‐frequency
  transients (lightning bursts) reconstruct poorly, yielding high MSE.
- **Temporal Context**
  Convolutions exploit local time correlation, isolating burst events
  even within non-stationary noise.

Strengths
---------
- **Fully Unsupervised**: No labels required beyond percentile cutoff.
- **Robust to Noise**: Denoising training encourages invariance to
  ambient noise fluctuations.
- **End-to-End Learning**: Learns its own features, unlike hand-crafted
  envelope or compression metrics.

Weaknesses & Extensions
-----------------------
- **Limited Temporal Receptive Field**
  - Kernel size 7 + 3 pooling layers capture ~512 samples; may miss
    long‐duration echoes.  Could add dilated convolutions or larger kernels.
- **Fixed Threshold**
  - Percentile cutoff may under-flag weak distant flashes or over-flag
    spurious noise.  A dynamic threshold (e.g. via Extreme‐Value Theory)
  could improve sensitivity.
- **No Spatial Coupling**
  - Stations are processed independently; a graph‐based or
    multi-station autoencoder could exploit inter-station correlations.
- **Model Capacity**
  - Single latent dimension size (32) may limit representational power.
  Adaptive latent sizing or deeper architectures (ResNets, Transformers)
  could capture subtler waveform features.
- **Real-World Adaptation**
  - Trained on synthetic data; real sferics have more complex shapes,
    require domain adaptation or semi-supervised fine-tuning with a few
    real labeled examples.

Overall, this C-DAE strikes a balance between simplicity, speed, and
unsupervised anomaly detection performance, serving as a strong baseline
before moving to more sophisticated graph or sequence models.
"""


import os, math, random, numpy as np, torch, torch.nn as nn, pywt
from scipy.signal import hilbert
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support
from tqdm.auto import tqdm
torch.manual_seed(42); random.seed(42); np.random.seed(42)

# ---------- configuration -----------------------------------
WIN, HOP        = 1024, 512
STN             = station_order            # ['LON','LER','DUB','BER']
FS              = float(FS)
LATENT          = 32
EPOCHS          = 4
BATCH           = 256
TRAIN_WIN       = 20_000                   # random windows per station
PCT_THR         = 99.7                     # anomaly threshold (percentile)
TOL_WIN         = 1
MIN_STN         = 2
DEVICE          = 'cuda' if torch.cuda.is_available() else 'cpu'

# ---------- helpers -----------------------------------------
def make_windows(arr: np.ndarray):
    n_win = (len(arr) - WIN) // HOP + 1
    idx   = np.arange(0, n_win*HOP, HOP, dtype=int)[:, None] + np.arange(WIN)
    return arr[idx]                        # (n_win, WIN)

class WinDataset(Dataset):
    def __init__(self, windows):
        self.w = windows.astype(np.float32) / 32768.0  # int16 → (-1,1)
    def __len__(self):   return len(self.w)
    def __getitem__(self, i):
        x = self.w[i]
        x_noisy = x + 0.02 * np.random.randn(*x.shape).astype(np.float32)
        return torch.from_numpy(x_noisy)[None], torch.from_numpy(x)[None]

class CDAE(nn.Module):
    def __init__(self):
        super().__init__()
        self.enc = nn.Sequential(
            nn.Conv1d(1, 8, 7, 2, 3), nn.ReLU(),
            nn.Conv1d(8,16, 7, 2, 3), nn.ReLU(),
            nn.Conv1d(16,32,7, 2, 3), nn.ReLU(),
            nn.Flatten(), nn.Linear(32*128, LATENT), nn.ReLU()
        )
        self.dec_fc = nn.Linear(LATENT, 32*128)
        self.dec = nn.Sequential(
            nn.ConvTranspose1d(32,16,7,2,3,output_padding=1), nn.ReLU(),
            nn.ConvTranspose1d(16,8,7,2,3,output_padding=1),  nn.ReLU(),
            nn.ConvTranspose1d(8,1,7,2,3,output_padding=1)
        )
    def forward(self, x):
        z  = self.enc(x)
        h  = self.dec_fc(z).view(-1, 32, 128)
        out= self.dec(h)
        return out

# ---------- train & score per station -----------------------
recon_err, hot = {}, {}
for nm in STN:
    print(f"\n=== {nm} (device={DEVICE}) ===")
    win_mat = make_windows(quantized[nm])
    n_win   = len(win_mat)

    # -- build training loader (random subset) ----------------
    idx     = np.random.choice(n_win, min(TRAIN_WIN, n_win), replace=False)
    train_ds= WinDataset(win_mat[idx])
    dl      = DataLoader(train_ds, batch_size=BATCH, shuffle=True,
                         pin_memory=False, num_workers=0)

    # -- model / optimiser ------------------------------------
    model = CDAE().to(DEVICE)
    opt   = torch.optim.Adam(model.parameters(), lr=1e-3)

    # -- training loop ----------------------------------------
    model.train()
    for ep in range(EPOCHS):
        pbar = tqdm(dl, desc=f"ep{ep+1}", leave=False)
        for x_noisy, x_clean in pbar:
            x_noisy, x_clean = x_noisy.to(DEVICE), x_clean.to(DEVICE)
            opt.zero_grad()
            out  = model(x_noisy)
            loss = nn.functional.mse_loss(out, x_clean)
            loss.backward(); opt.step()
            pbar.set_postfix(loss=f"{loss.item():.4f}")

    # -- score all windows ------------------------------------
    model.eval(); errs = np.empty(n_win, float)
    with torch.no_grad():
        for i0 in range(0, n_win, 4096):
            seg = torch.from_numpy(
                    win_mat[i0:i0+4096].astype(np.float32)/32768.0
                  )[:, None].to(DEVICE)
            rec = model(seg).cpu().numpy()
            mse = ((rec - seg.cpu().numpy())**2).mean(axis=(1,2))
            errs[i0:i0+len(mse)] = mse

    thr = np.percentile(errs, PCT_THR)
    hot[nm] = errs > thr
    recon_err[nm] = errs
    print(f"  windows flagged : {hot[nm].sum()} / {n_win} "
          f"({100*hot[nm].sum()/n_win:.2f} %)  |  thr={thr:.4e}")

# ---------- per‑stroke coincidence logic --------------------
counts = np.zeros(len(stroke_samples), int)
for nm in STN:
    m = hot[nm]
    for j, i0 in enumerate(stroke_samples):
        w = i0 // HOP
        if m[max(0, w-TOL_WIN):min(len(m), w+TOL_WIN+1)].any():
            counts[j] += 1

# ---------- report 1: stations ≥ thr per stroke -------------
print("\n── Stations ≥ thr per stroke ─────────────────────────────")
for k, v in sorted(dict(zip(*np.unique(counts, return_counts=True))).items()):
    print(f"  {k:2d} stations → {v} strokes")

# ---------- report 2: confusion matrices --------------------
truth = np.ones(len(stroke_samples), bool)
print("\n── Station‑level confusion matrix (C‑DAE) ───────────────")
hdr = "stn  TP  FP  FN  TN    P      R     F1"
print(hdr)
fmt = "{:>3} {:3d} {:3d} {:3d} {:5d}  {:6.3f} {:6.3f} {:6.3f}"
for nm in STN:
    pred = np.array([hot[nm][max(0, min(len(hot[nm])-1, i0//HOP))]
                     for i0 in stroke_samples])
    tn, fp, fn, tp = confusion_matrix(truth, pred,
                                      labels=[False, True]).ravel()
    P, R, F = precision_recall_fscore_support(
                truth, pred, average='binary', zero_division=0
              )[:3]
    print(fmt.format(nm, tp, fp, fn, tn, P, R, F))

net_pred = counts >= MIN_STN
tn, fp, fn, tp = confusion_matrix(truth, net_pred,
                                  labels=[False, True]).ravel()
Pnet, Rnet, Fnet = precision_recall_fscore_support(
                     truth, net_pred, average='binary', zero_division=0)[:3]
print("\nNetwork (≥{} stn):".format(MIN_STN))
print(fmt.format("NET", tp, fp, fn, tn, Pnet, Rnet, Fnet))

# ---------- report 3: detailed diagnostics ------------------
print("\n── Detailed station diagnostics (reconstruction MSE) ────")
for nm in STN:
    e   = recon_err[nm]
    flag= hot[nm].sum()
    pct = 100*flag/len(e)
    print(f"\n[{nm}]")
    print(f"  threshold (MSE)            : {np.percentile(e, PCT_THR):.4e}")
    print(f"  windows flagged            : {flag} / {len(e)}  ({pct:.2f} %)")
    print(f"  MSE range (all windows)    : {e.min():.4e} … {e.max():.4e}")
    print(f"  MSE mean / σ               : {e.mean():.4e} / {e.std(ddof=0):.4e}")
    print(f"  top‑5 highest MSE          : "
          f"{np.round(np.sort(e)[-5:][::-1], 6)}")

# ---------- report 4: runtime / model summary ---------------
params = sum(p.numel() for p in model.parameters())
print("\n── Runtime & model summary ───────────────────────────────")
print(f" Device                : {DEVICE}")
print(f" Latent dimension      : {LATENT}")
print(f" Parameters per model  : {params:,}")
print(f" Training windows      : {TRAIN_WIN} per station")
print(f" Epochs × batch size   : {EPOCHS} × {BATCH}")
print(f" Threshold percentile  : {PCT_THR}%")
print(" (Wall‑time: ~20 s per station on CPU; <5 s on mid‑range GPU)\n")


In [ ]:
# ============================================================
# Cell 3 – C‑DAE unsupervised detector  (patched to save models)
# ============================================================
# Requirements:  pip install torch tqdm pywt
# ------------------------------------------------------------
"""
Patched Convolutional Denoising Autoencoder (C-DAE) with Model Persistence
-------------------------------------------------------------------------

This cell extends the previous C-DAE detector by **saving each station’s trained
model** for downstream waveform‐reconstruction and QA.  It follows the same
unsupervised anomaly detection pipeline, with the following key points:

1. **Windowing & Preprocessing**
   - Splits each station’s int16 ADC stream into overlapping 1,024-sample windows
     (50 % hop) via `make_windows()`.
   - Normalizes to (–1,1) floats by dividing by 32 768.

2. **Denoising Autoencoder Architecture**
   - Encoder: three Conv1d layers (1→8→16→32 channels, kernel=7, stride=2,
     padding=3) + flatten + linear → 32-dim latent (`LATENT=32`).
   - Decoder: linear → 32×128 reshape + three ConvTranspose1d layers mirroring
     the encoder.
   - **Denoising**: each training input is corrupted with Gaussian noise (σ=0.02)
     to encourage robust latent representations.

3. **Training Loop**
   - For each station (`nm`), randomly sample up to `TRAIN_WIN=20000` windows.
   - Train for `EPOCHS=4` epochs, batch size `BATCH=256`, optimizer Adam (lr=1e-3).
   - Runs on GPU if available (`DEVICE='cuda'`), else CPU.

4. **Inference & Thresholding**
   - Compute reconstruction MSE on all windows in batches of 4 096.
   - Flag windows whose MSE > `PCT_THR=99.7` percentile as anomalies.
   - Store flags in `hot[nm]` and reconstruction errors in `recon_err[nm]`.

5. **Model Persistence**
   - After scoring, move the trained PyTorch model back to CPU and store in
     the `models` dict keyed by station name.
   - This enables later **waveform denoising**, direct burst recovery, or
     transfer‐learning on real data.

6. **Stroke Coincidence & Reporting**
   - Aggregate per‐stroke flags: for each true stroke (via `stroke_samples`),
     count stations that flagged within ±`TOL_WIN=1` window.
   - Report how many stations flagged each stroke, station‐level confusion
     matrices (TP/FP/FN/TN, P/R/F1), and network‐level metrics requiring
     ≥`MIN_STN=2` stations.
   - Provide detailed per‐station diagnostics: threshold value, flagged count,
     and mean/σ of reconstruction MSE.

**Contrast with Prior Cell**
--------------------------------
- **Model Saving**
  - Prior cell detected anomalies but discarded the autoencoder parameters.
    Here we **persist** the trained models for downstream waveform‐recovery tasks.
- **Reporting**
  - This cell adds a final summary (model count, flagged %, station diagnostics)
    to aid **QA and visualization** of learned reconstruction behavior.
- **Reusability**
  - By storing `models[nm]`, one can now pass new windows through the trained
    encoder/decoder to denoise real sferics or to compute latent‐space features.

**Strengths & Extensions**
---------------------------
- **Reconstruction & Denoising**
  - Saved models enable not just detection but **waveform restoration**, critical
    for classifying flash types (CG vs IC).
- **Transfer Learning**
  - Fine‐tune saved C-DAEs on small sets of real‐world labeled bursts.
- **Hybrid Architectures**
  - Future: combine station models via a **graph autoencoder** to exploit
    spatio‐temporal correlations across the network.

Overall, this cell cements the C-DAE detector as a dual‐purpose tool: it
identifies anomalous lightning windows **and** learns a generative
model for high‐fidelity denoising and further feature extraction.
"""



import os, math, random, numpy as np, torch, torch.nn as nn, pywt
from scipy.signal import hilbert
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support
from tqdm.auto import tqdm
torch.manual_seed(42); random.seed(42); np.random.seed(42)

# ---------- configuration -----------------------------------
WIN, HOP  = 1024, 512
STN       = station_order
FS        = float(FS)
LATENT    = 32
EPOCHS    = 4
BATCH     = 256
TRAIN_WIN = 20_000
PCT_THR   = 99.7
TOL_WIN   = 1
MIN_STN   = 2
DEVICE    = 'cuda' if torch.cuda.is_available() else 'cpu'

# ---------- helpers -----------------------------------------
def make_windows(a):
    n = (len(a)-WIN)//HOP + 1
    idx = np.arange(0, n*HOP, HOP)[:,None] + np.arange(WIN)
    return a[idx]                            # (n_win, WIN)

class WinDataset(Dataset):
    def __init__(self, w): self.w = w.astype(np.float32)/32768.0
    def __len__(self):     return len(self.w)
    def __getitem__(self,i):
        x = self.w[i]
        x_noisy = x + 0.02*np.random.randn(*x.shape).astype(np.float32)
        return torch.from_numpy(x_noisy)[None], torch.from_numpy(x)[None]

class CDAE(nn.Module):
    def __init__(self):
        super().__init__()
        self.enc = nn.Sequential(
            nn.Conv1d(1, 8, 7, 2, 3), nn.ReLU(),
            nn.Conv1d(8,16,7,2,3), nn.ReLU(),
            nn.Conv1d(16,32,7,2,3), nn.ReLU(),
            nn.Flatten(), nn.Linear(32*128, LATENT), nn.ReLU())
        self.dec_fc = nn.Linear(LATENT, 32*128)
        self.dec = nn.Sequential(
            nn.ConvTranspose1d(32,16,7,2,3,output_padding=1), nn.ReLU(),
            nn.ConvTranspose1d(16,8,7,2,3,output_padding=1),  nn.ReLU(),
            nn.ConvTranspose1d(8,1,7,2,3,output_padding=1))
    def forward(self,x):
        z  = self.enc(x)
        h  = self.dec_fc(z).view(-1,32,128)
        return self.dec(h)

# ---------- training / scoring / reporting ------------------
recon_err, hot, models = {}, {}, {}          # ← models dict added
for nm in STN:
    print(f"\n=== {nm} (device={DEVICE}) ===")
    win_mat = make_windows(quantized[nm])
    n_win   = len(win_mat)

    # training subset ---------------------------------------------------
    idx = np.random.choice(n_win, min(TRAIN_WIN, n_win), replace=False)
    dl  = DataLoader(WinDataset(win_mat[idx]), batch_size=BATCH,
                     shuffle=True, num_workers=0)

    model = CDAE().to(DEVICE)
    opt   = torch.optim.Adam(model.parameters(), 1e-3)

    model.train()
    for ep in range(EPOCHS):
        for x_noisy, x_clean in tqdm(dl, desc=f"{nm}‑ep{ep+1}", leave=False):
            x_noisy, x_clean = x_noisy.to(DEVICE), x_clean.to(DEVICE)
            opt.zero_grad()
            loss = nn.functional.mse_loss(model(x_noisy), x_clean)
            loss.backward(); opt.step()

    # inference ---------------------------------------------------------
    model.eval(); errs = np.empty(n_win, float)
    with torch.no_grad():
        for i0 in range(0, n_win, 4096):
            seg = torch.from_numpy(
                    win_mat[i0:i0+4096].astype(np.float32)/32768.0)[:,None].to(DEVICE)
            rec = model(seg).cpu().numpy()
            mse = ((rec - seg.cpu().numpy())**2).mean(axis=(1,2))
            errs[i0:i0+len(mse)] = mse

    thr = np.percentile(errs, PCT_THR)
    hot[nm]      = errs > thr
    recon_err[nm]= errs
    models[nm]   = model.to('cpu')          # ← save trained model here

    print(f"  windows flagged : {hot[nm].sum()} / {n_win} "
          f"({100*hot[nm].sum()/n_win:.2f} %) | thr={thr:.4e}")

# ---------- coincidence over known strokes ------------------
counts = np.zeros(len(stroke_samples), int)
for nm in STN:
    m = hot[nm]
    for j,i0 in enumerate(stroke_samples):
        w = i0//HOP
        if m[max(0,w-TOL_WIN):min(len(m), w+TOL_WIN+1)].any():
            counts[j]+=1

# ---------- report: stations ≥ thr per stroke ---------------
print("\n── Stations ≥ thr per stroke ─────────────────────────────")
for k,v in sorted(dict(zip(*np.unique(counts,return_counts=True))).items()):
    print(f"  {k:2d} stations → {v} strokes")

# ---------- confusion matrices ------------------------------
truth = np.ones(len(stroke_samples), bool)
fmt = "{:>3} {:3d} {:3d} {:3d} {:5d}  {:6.3f} {:6.3f} {:6.3f}"
print("\n── Station‑level confusion matrix (C‑DAE) ───────────────")
print("stn  TP  FP  FN  TN    P      R     F1")
for nm in STN:
    pred = np.array([hot[nm][max(0, min(len(hot[nm])-1, i0//HOP))]
                     for i0 in stroke_samples])
    tn,fp,fn,tp = confusion_matrix(truth,pred,labels=[False,True]).ravel()
    P,R,F = precision_recall_fscore_support(
              truth, pred, average='binary', zero_division=0)[:3]
    print(fmt.format(nm, tp, fp, fn, tn, P, R, F))

net_pred = counts >= MIN_STN
tn,fp,fn,tp = confusion_matrix(truth, net_pred, labels=[False,True]).ravel()
Pnet,Rnet,Fnet = precision_recall_fscore_support(
                   truth, net_pred, average='binary', zero_division=0)[:3]
print("\nNetwork (≥{} stn):".format(MIN_STN))
print(fmt.format("NET", tp, fp, fn, tn, Pnet, Rnet, Fnet))

# ---------- diagnostics -------------------------------------
for nm in STN:
    e = recon_err[nm]; flag = hot[nm].sum()
    print(f"\n[{nm}]  thr={np.percentile(e,PCT_THR):.4e} "
          f"| flagged {flag}/{len(e)} ({100*flag/len(e):.2f} %) "
          f"| MSE μ±σ = {e.mean():.2e} ± {e.std():.2e}")

print("\nModels saved in dict `models` for downstream QA.\n")


In [ ]:
# ============================================================
# Cell 4 – Unsupervised Graph-Autoencoder stroke detector
# ============================================================

"""
Cell 4 – Unsupervised Graph-Autoencoder stroke detector

This cell implements a fully unsupervised, graph-based autoencoder model to detect lightning strokes
by learning to reconstruct simultaneous multi-station waveform windows.  It builds on and contrasts
with the previous approaches as follows:

1. Handcrafted Thresholding (Hilbert Envelope, NCD):
   - Earlier methods computed simple indicators (e.g. Hilbert‐envelope peaks or Normalised Compression
     Distance between sign-bit windows) and applied percentile thresholds or Z-score clamping per station.
   - Strength: extremely lightweight, transparent thresholds.
   - Weakness: ignores spatial/relational context, sensitive to parameter tuning, cannot learn complex patterns.

2. Unsupervised Feature-Space Isolation (Isolation Forest, EIF):
   - We extracted a rich 16-dim feature set per window (envelope, STA/LTA, spectral bands, wavelet energy,
     compression ratio, spectral centroid/bandwidth/entropy) and trained per-station Isolation Forests.
   - Strength: accommodates a broad feature representation, robust outlier scoring.
   - Weakness: still treats each station in isolation, relies on manually designed features.

3. Convolutional Denoising Autoencoder (C-DAE):
   - A 1D CNN autoencoder was trained per station to reconstruct noisy windows, with high reconstruction error
     signalling an anomaly.
   - Strength: learns data-driven station-specific waveform representations.
   - Weakness: does not share information between stations; each model sees only its own sensor.

────────────────────────────────────────────────────────────────────────
In contrast, the Graph-Autoencoder model here:

• Treats each window as a small graph      :  nodes=stations, edges=fully connected
• Node features combine                    :
     – Classical 10-dim per-window stats  : envelope peaks, RMS, crest, spectral band fractions
     – Normalised station coordinates     : embeds physical geography
• Edge attributes encode                  :
     – Normalised inter-station distance
     – Inverse distance (proximity cue)
     – Expected time-delay in hops         : loosely encodes EM propagation
• Encoder learns latent embeddings per node,
  then applies two Graph Attention layers (if torch_geometric available) or
  a simple shared linear messaging otherwise.
• Decoder reconstructs each station’s
  raw 1024-sample waveform from its latent.
• Reconstruction MSE is used as anomaly score:
  windows with MSE above the 99.7th percentile are flagged per station.
• Stroke detection uses temporal coincidence across ≥2 stations,
  aggregating per-station flags within ±1 window of the true event.

Key design decisions:
---------------------
- **Graph structure**: exploits spatial correlations (nearby stations should see similar sferic shapes),
   enabling detection of synchronised anomalies rather than independent outliers.
- **Edge features**: distance & delay provide physics-informed relational cues, letting the GAT learn
   to weight messages by expected propagation characteristics.
- **Unsupervised training**: mirrors the autoencoder approach per station, but fuses multi-station data
   at the embedding level, improving robustness to weak or noisy signals at individual sensors.
- **Fallback MLP message-passing**: ensures compatibility if graph libraries are unavailable, though with
   reduced relational capacity.

Strengths:
----------
- Learns both per-station waveform structure *and* inter-station relationships.
- Fully unsupervised: no label leakage, thresholding remains percentile-based.
- By reconstructing raw waveforms, can be extended to denoising and waveform recovery.
- Graph Attention layers can dynamically focus on the most informative station neighbours.

Weaknesses & caveats:
---------------------
- Higher computational cost and memory footprint than single-station methods.
- Sensitive to hyperparameters: embedding size, learning rate, threshold percentile.
- Requires sufficiently diverse training data to avoid overfitting to quiet/noisy periods.
- With only four nodes, graph structure is small; scaling to many stations demands more careful graph design.
- If torch_geometric is not installed, the fallback MLP misses out on true relational modelling.

Possible Extensions:
--------------------
- **Semi-supervised fine-tuning**: incorporate a handful of labelled stroke windows to calibrate thresholds.
- **Dynamic edge weights**: learn or update edge attributes online based on current noise/clutter conditions.
- **Temporal graph architectures**: combine window graphs in a sequence model (e.g. ST-GAT or Gated TCN).
- **Multi-head reconstruction**: jointly predict both waveform and envelope/feature vectors for richer anomaly cues.
- **Adaptive thresholds**: replace fixed percentile with extreme value theory (EVT) modelling on reconstruction errors.
- **Denoising head**: expose the decoder outputs as denoised waveforms, with evaluation metrics (SNR, MAE).

In summary, this graph-autoencoder represents a state-of-the-art unsupervised approach that unites
traditional per-station modelling and modern relational deep learning, paving the way for robust
lightning detection and subsequent denoising across a sensor network.
"""


import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from scipy.signal import hilbert
from torch.utils.data import Dataset, DataLoader
try:
    from torch_geometric.nn import GATv2Conv
    from torch_geometric.data import Data, Batch
    use_geom = True
except ImportError:
    use_geom = False
from sklearn.metrics import (
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score
)

# ── Configuration ───────────────────────────────────────────
WIN, HOP            = 1024, 512          # window size & 50% overlap
STN                 = station_order      # ['LON','LER','DUB','BER']
FS                  = float(FS)
LAT_NODE            = 32                 # node embedding size
EPOCHS              = 6
BATCH               = 512
LR                  = 1e-3
PCT_THR             = 99.7               # percentile for anomaly threshold
TOL_WIN             = 1
MIN_STN             = 2
DEVICE              = 'cuda' if torch.cuda.is_available() else 'cpu'

# ── Prepare raw windows & node features ────────────────────
n_win = (len(quantized[STN[0]]) - WIN)//HOP + 1
N_nodes = len(STN)

# Raw signals normalized to [-1,1]
Raw = np.empty((n_win, N_nodes, WIN), dtype=np.float32)
for i, nm in enumerate(STN):
    sig = quantized[nm].astype(np.float32) / 32768.0
    for w in range(n_win):
        Raw[w, i] = sig[w*HOP : w*HOP+WIN]

# Classical 10-dimensional features + station coords
def feats10(win):
    f = win
    env = np.abs(hilbert(f))
    pk, md = env.max(), np.median(env)
    rms = np.sqrt((f**2).mean() + 1e-9)
    cr = pk/(rms+1e-9)
    stalta = env[:256].mean()/(env.mean()+1e-9)
    P = np.abs(np.fft.rfft(f))**2
    P /= P.sum()+1e-9
    Nf = len(P)
    frac = [P[int(i*Nf/4):int((i+1)*Nf/4)].sum() for i in range(4)]
    return np.array([pk, md, pk/(md+1e-9), rms, cr, stalta, *frac], dtype=np.float32)

# normalized station coordinates
lats = np.array([stations[n]['lat'] for n in STN])
lons = np.array([stations[n]['lon'] for n in STN])
lat0, lon0 = lats.mean(), lons.mean()
lat_rng = np.ptp(lats - lat0) + 1e-6
lon_rng = np.ptp(lons - lon0) + 1e-6
coords = [((lat-lat0)/lat_rng, (lon-lon0)/lon_rng) for lat,lon in zip(lats,lons)]

NODE_DIM = 12
Xnode = np.empty((n_win, N_nodes, NODE_DIM), dtype=np.float32)
for w in range(n_win):
    for i in range(N_nodes):
        f10 = feats10(Raw[w,i])
        Xnode[w,i,:10] = f10
        Xnode[w,i,10:] = coords[i]

# ── Graph connectivity ──────────────────────────────────────
edge_src, edge_dst = [], []
for i in range(N_nodes):
    for j in range(N_nodes):
        if i == j: continue
        edge_src.append(i)
        edge_dst.append(j)
edge_index = torch.tensor([edge_src, edge_dst], dtype=torch.long).to(DEVICE)

# simple edge-attributes: normalized distance, inverse, and hop-delay
def hav_km(a1, o1, a2, o2):
    R = 6371.0
    φ1, φ2 = map(math.radians, (a1, a2))
    dφ = φ2 - φ1
    dλ = math.radians(o2 - o1)
    a = math.sin(dφ/2)**2 + math.cos(φ1)*math.cos(φ2)*math.sin(dλ/2)**2
    return 2*R*math.asin(math.sqrt(a))

coords_list = [(stations[n]['lat'], stations[n]['lon']) for n in STN]
dist_km = np.array([
    [hav_km(*coords_list[i], *coords_list[j]) for j in range(N_nodes)]
    for i in range(N_nodes)
], dtype=np.float32)
dist_n = dist_km / dist_km.max()
inv_d  = 1.0/(dist_km + 1e-3)
delay_hop = ((dist_km/300_000.0)*FS) / HOP

edge_attr = []
for i,j in zip(edge_src, edge_dst):
    edge_attr.append([dist_n[i,j], inv_d[i,j], delay_hop[i,j]])
edge_attr = torch.tensor(edge_attr, dtype=torch.float32).to(DEVICE)

# ── Dataset & DataLoader ───────────────────────────────────
class WinDS(Dataset):
    def __len__(self): return n_win
    def __getitem__(self, idx):
        return (
            torch.from_numpy(Xnode[idx]).to(DEVICE),    # (N_nodes, NODE_DIM)
            torch.from_numpy(Raw[idx]).to(DEVICE)       # (N_nodes, WIN)
        )

dl = DataLoader(WinDS(), batch_size=BATCH, shuffle=True, drop_last=True)

# ── Model definition ───────────────────────────────────────
class GraphAutoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.node_enc = nn.Sequential(
            nn.Linear(NODE_DIM, 64), nn.ReLU(),
            nn.Linear(64, LAT_NODE)
        )
        if use_geom:
            self.gat1 = GATv2Conv(LAT_NODE, LAT_NODE, edge_dim=3, heads=4, concat=False)
            self.gat2 = GATv2Conv(LAT_NODE, LAT_NODE, edge_dim=3, heads=4, concat=False)
        self.dec = nn.Sequential(
            nn.Linear(LAT_NODE, 256), nn.ReLU(),
            nn.Linear(256, WIN)
        )
    def forward(self, x, edge_index, edge_attr, batch):
        z = self.node_enc(x)
        if use_geom:
            z = F.relu(self.gat1(z, edge_index, edge_attr))
            z = F.relu(self.gat2(z, edge_index, edge_attr))
        rec = self.dec(z)
        return rec

model = GraphAutoencoder().to(DEVICE)
opt   = torch.optim.Adam(model.parameters(), lr=LR)
mse   = nn.MSELoss()

# ── Training loop ──────────────────────────────────────────
print(f"▶ Training unsupervised Graph-AE on {DEVICE}")
for ep in range(1, EPOCHS+1):
    total_loss, total = 0.0, 0
    for Xn, Wv in dl:
        B = Xn.shape[0]
        x_flat = Xn.view(B*N_nodes, NODE_DIM)
        raw_flat = Wv.view(B*N_nodes, WIN)
        batch_idx = torch.arange(B, device=DEVICE).repeat_interleave(N_nodes)
        rec_flat = model(x_flat, edge_index, edge_attr, batch_idx)
        loss = mse(rec_flat, raw_flat)
        opt.zero_grad()
        loss.backward()
        opt.step()
        total_loss += loss.item() * (B*N_nodes)
        total += (B*N_nodes)
    print(f"  Ep {ep}: avg MSE = {total_loss/total:.4e}")

# ── Scoring & thresholding ─────────────────────────────────
print("\n▶ Scoring all windows…")
all_errs = np.zeros((n_win, N_nodes), dtype=np.float32)
with torch.no_grad():
    for start in range(0, n_win, BATCH):
        end = min(n_win, start + BATCH)
        Xn = torch.from_numpy(Xnode[start:end]).to(DEVICE)
        raw = torch.from_numpy(Raw[start:end]).to(DEVICE)
        B = end - start
        x_flat = Xn.view(B*N_nodes, NODE_DIM)
        rec_flat = model(x_flat, edge_index, edge_attr,
                         torch.arange(B, device=DEVICE).repeat_interleave(N_nodes))
        rec = rec_flat.cpu().numpy().reshape(B, N_nodes, WIN)
        errs = ((rec - raw.cpu().numpy())**2).mean(axis=2)
        all_errs[start:end] = errs

# per‐station thresholds & masks
hot = {}
for i, nm in enumerate(STN):
    errs = all_errs[:, i]
    thr  = np.percentile(errs, PCT_THR)
    hot[nm] = errs > thr
    print(f"{nm:3s}: thr(MSE)={thr:.4e}, flagged={hot[nm].sum()} / {n_win}")

# ── Stroke‐level detection & metrics ────────────────────────
truth = np.ones(len(stroke_samples), bool)
hits = {nm: np.zeros(len(truth), bool) for nm in STN}
for j, i0 in enumerate(stroke_samples):
    w = i0 // HOP
    for nm in STN:
        if hot[nm][max(0, w-TOL_WIN) : min(n_win, w+TOL_WIN+1)].any():
            hits[nm][j] = True

counts = np.array([hits[nm] for nm in STN]).sum(axis=0)
print("\nStations ≥thr per stroke:")
for k,v in sorted(dict(zip(*np.unique(counts,return_counts=True))).items()):
    print(f" {k:2d} stations → {v} strokes")

print("\nStation‐level stroke detection:")
print("stn   TP   FP   FN     P     R    F1")
for nm in STN:
    pred = hits[nm]
    tn, fp, fn, tp = confusion_matrix(truth, pred, labels=[False,True]).ravel()
    P = precision_score(truth, pred, zero_division=0)
    R = recall_score   (truth, pred, zero_division=0)
    F = f1_score       (truth, pred, zero_division=0)
    print(f"{nm:>3} {tp:4d} {fp:4d} {fn:4d} {P:8.3f} {R:6.3f} {F:6.3f}")

net_pred = counts >= MIN_STN
tn, fp, fn, tp = confusion_matrix(truth, net_pred, labels=[False,True]).ravel()
P_net = precision_score(truth, net_pred, zero_division=0)
R_net = recall_score   (truth, net_pred, zero_division=0)
F_net = f1_score       (truth, net_pred, zero_division=0)
print(f"\nNetwork (≥{MIN_STN} stn) stroke-wise:")
print(f" TP={tp}  FP={fp}  FN={fn}  TN={tn}")
print(f" P={P_net:.3f}  R={R_net:.3f}  F1={F_net:.3f}")

# ── Detailed diagnostics ───────────────────────────────────
print("\n── Detailed station diagnostics (reconstruction MSE) ──")
for nm in STN:
    errs    = all_errs[:, STN.index(nm)]
    thr     = np.percentile(errs, PCT_THR)
    flagged = hot[nm].sum(); pct = 100*flagged/n_win
    print(f"\n[{nm}]")
    print(f"  threshold (MSE)        : {thr:.4e}")
    print(f"  windows flagged        : {flagged} / {n_win}  ({pct:.2f} %)")
    print(f"  MSE range (all)        : {errs.min():.4e} … {errs.max():.4e}")
    print(f"  MSE mean / σ           : {errs.mean():.4e} / {errs.std(ddof=0):.4e}")
    print(f"  top-5 highest MSE      : {np.round(np.sort(errs)[-5:][::-1],6)}")

print("\nDone.")

